# Import the necessary Libraries

In [40]:
import pandas as pd
import numpy as np
import random
import requests
import time

## To start our analysis and to comply with compute restrictions I will first analyse the data for a total of 5 companies out of 500

I will first start by studying 5 companies in the same industry so to have a more industry specific insights. Given the companies in that industry I will know the patterns that emerge in that specific industry and from that point on, when I have completed that industry I can generalize it to other companies in the same industry at first and than into other industries

### Advantages:
1. **Domain-Specific patterns**
2. Comparable analysis
3. Controlled complexity
4. Faster iteration
5. Clear baseline

### Information Techonology Industry
1. **Microsoft** 
2. **ServiceNow**
3. **AMD**
4. **Salesforce**
5. **Palantir**

### I am downloading the MSFT financial data
- Import the Financial csv data
- sort values by date
- Pivot the table to make the concept appear as independant columns across different date&Time
- Given that during the pivote there were a lot of duplicate that were generated, I group the columns by filed date and remove any NaN cells as to have a cleaned dataframe
- To further clean the data, I removed the columns that are 95% empty. I cannot do any meaning full Time Series analysis on those columns

In [41]:
df = pd.read_csv('./data/financial_data/AMZN_raw_financials.csv')

In [42]:

pivoted_df = df.pivot_table(
    index=['cik', 'company_name', 'frame','unit', 'end', 'filed', 'form'],
    columns='concept',
    values='value'
).reset_index()

pivoted_df.head(2)

concept,cik,company_name,frame,unit,end,filed,form,AccountsPayable,AccountsPayableCurrent,AccountsReceivableNetCurrent,...,UnrecordedUnconditionalPurchaseObligationBalanceOnFourthAnniversary,UnrecordedUnconditionalPurchaseObligationBalanceOnSecondAnniversary,UnrecordedUnconditionalPurchaseObligationBalanceOnThirdAnniversary,UnrecordedUnconditionalPurchaseObligationBalanceSheetAmount,UnrecordedUnconditionalPurchaseObligationDueAfterFiveYears,UnrecordedUnconditionalPurchaseObligationDueInRemainderOfFiscalYear,ValuationAllowanceDeferredTaxAssetChangeInAmount,VariableLeaseCost,WeightedAverageNumberOfDilutedSharesOutstanding,WeightedAverageNumberOfSharesOutstandingBasic
0,1018724,"AMAZON.COM, INC.",CY2006Q4I,USD,2006-12-31,2010-01-29,10-K,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1018724,"AMAZON.COM, INC.",CY2007,USD,2007-12-31,2010-01-29,10-K,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Code that groups by filed column and remove most of the NaN cells. This way I will have all the relavant columns and remove any duplicates and NaN values**

In [43]:
# Define metadata columns to preserve
metadata_cols = ['cik', 'company_name', 'form', 'frame', 'end', 'unit']

# Create an empty DataFrame to store results
consolidated_df = pd.DataFrame()

# Group by filed date
grouped = pivoted_df.groupby('end')

# Iterate through each group
for filed_date, group in grouped:
    # Create a dictionary to store the row data
    row_data = {'end': filed_date}
    
    # For metadata columns, just take the first value
    for col in metadata_cols:
        if col in pivoted_df.columns:
            row_data[col] = group[col].iloc[0]
    
    # For all other columns, take the first non-NaN value
    for col in pivoted_df.columns:
        if col not in metadata_cols and col != 'end':
            non_nan_values = group[col].dropna()
            if not non_nan_values.empty:
                row_data[col] = non_nan_values.iloc[0]
            else:
                row_data[col] = None
    
    # Append this row to our result DataFrame
    consolidated_df = pd.concat([consolidated_df, pd.DataFrame([row_data])], ignore_index=True)

# Display the first few rows to verify
consolidated_df.head()

/var/folders/0w/q70p80s12nz5lpzr087614lw0000gn/T/ipykernel_13498/41078436.py:30: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  consolidated_df = pd.concat([consolidated_df, pd.DataFrame([row_data])], ignore_index=True)


,end,cik,company_name,form,frame,unit,filed,AccountsPayable,AccountsPayableCurrent,AccountsReceivableNetCurrent,...,UnrecordedUnconditionalPurchaseObligationBalanceOnFourthAnniversary,UnrecordedUnconditionalPurchaseObligationBalanceOnSecondAnniversary,UnrecordedUnconditionalPurchaseObligationBalanceOnThirdAnniversary,UnrecordedUnconditionalPurchaseObligationBalanceSheetAmount,UnrecordedUnconditionalPurchaseObligationDueAfterFiveYears,UnrecordedUnconditionalPurchaseObligationDueInRemainderOfFiscalYear,ValuationAllowanceDeferredTaxAssetChangeInAmount,VariableLeaseCost,WeightedAverageNumberOfDilutedSharesOutstanding,WeightedAverageNumberOfSharesOutstandingBasic
0,2006-12-31,1018724,"AMAZON.COM, INC.",10-K,CY2006Q4I,USD,2010-01-29,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2007-06-30,1018724,"AMAZON.COM, INC.",10-Q,CY2007Q2I,USD,2009-07-24,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2007-09-30,1018724,"AMAZON.COM, INC.",10-Q,CY2007Q3I,USD,2009-10-23,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2007-12-31,1018724,"AMAZON.COM, INC.",10-K,CY2007,USD,2010-01-29,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,424000000.0,413000000.0
4,2008-03-31,1018724,"AMAZON.COM, INC.",10-Q,CY2008Q1I,USD,2010-04-23,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
# Filter the DataFrame to keep only rows where year in 'filed' column is 2020 or later
consolidated_df = consolidated_df[consolidated_df['filed'].str[:4].astype(int) >= 2019]
# Keep rows with exactly Q1, Q2, Q3, Q4 or just the year (like CY2019)
consolidated_df = consolidated_df[~consolidated_df['frame'].str.match(r'CY\d{4}Q[1-4].+')]

**In the following function I am computing the number of empty cells in each columns and removing all the columns that are 95% empty**

In [45]:
# Calculate the percentage of NaN values in each column
nan_percentages = consolidated_df.isna().mean() * 100

# Identify columns with 95% or more NaN values
columns_to_drop = nan_percentages[nan_percentages >= 85].index.tolist()

# Display columns that will be dropped
print(f"Columns being dropped (>= 95% empty): {columns_to_drop}")
print(f"Number of columns before: {consolidated_df.shape[1]}")

# Drop the identified columns
consolidated_df = consolidated_df.drop(columns=columns_to_drop)

# Display the new shape
print(f"Number of columns after: {consolidated_df.shape[1]}")

Columns being dropped (>= 95% empty): ['AccountsPayable', 'AccruedLiabilities', 'AccruedLiabilitiesForUnredeeemedGiftCards', 'AccumulatedDepreciationDepletionAndAmortizationPropertyPlantAndEquipment', 'AccumulatedOtherComprehensiveIncomeLossAvailableForSaleSecuritiesAdjustmentNetOfTax', 'AccumulatedOtherComprehensiveIncomeLossForeignCurrencyTranslationAdjustmentNetOfTax', 'AcquiredFiniteLivedIntangibleAssetWeightedAverageUsefulLife', 'AdjustmentsNoncashItemsToReconcileNetIncomeLossToCashProvidedByUsedInOperatingActivitiesOther', 'AdjustmentsToAdditionalPaidInCapitalTaxEffectFromShareBasedCompensation', 'AllowanceForDoubtfulAccountsReceivable', 'AmortizationOfAcquiredIntangibleAssets', 'AmortizationOfLeasedAsset', 'AntidilutiveSecuritiesExcludedFromComputationOfEarningsPerShareAmount', 'AssetImpairmentCharges', 'AvailableForSaleDebtSecuritiesAmortizedCostBasis', 'AvailableForSaleSecuritiesDebtSecurities', 'AvailableForSaleSecuritiesGrossRealizedGains', 'AvailableForSaleSecuritiesGrossRe

# Balance Sheet for 10-K

In [46]:
import pandas as pd
import numpy as np

def reconstruct_balance_sheet(df):
    """
    Reconstructs balance sheet from XBRL-tagged data.
    
    Args:
        df: DataFrame containing XBRL-tagged financial data
        
    Returns:
        DataFrame with reconstructed balance sheet for each period
    """
    # Create a new DataFrame to store the reconstructed balance sheet
    balance_sheet = pd.DataFrame(index=df.index)
    
    # Copy identifying columns
    id_columns = ['filed', 'company_name', 'end', 'unit', 'form', 'frame', 'cik']
    for col in id_columns:
        if col in df.columns:
            balance_sheet[col] = df[col]
    
    # Get all available columns for matching
    available_columns = set(df.columns)
    
    # Define mappings for balance sheet accounts based on XBRL structure
    
    # Assets section
    assets = {
        # Current Assets
        'Cash and Cash Equivalents': {
            'primary': 'CashAndCashEquivalents',
            'alternatives': ['CashAndCashEquivalentsAtCarryingValue'],
            'children': []
        },
        'Short-term Investments': {
            'primary': 'ShortTermInvestments',
            'alternatives': ['MarketableSecuritiesCurrent'],
            'children': ['AvailableForSaleSecuritiesCurrent', 'InvestmentsAvailableForSaleCurrent']
        },
        'Accounts Receivable, Net (Current)': {
            'primary': 'AccountsReceivableNetCurrent',
            'alternatives': [],
            'children': []
        },
        'Accounts Receivable, Net (Non-current)': {
            'primary': 'AccountsReceivableNetNoncurrent',
            'alternatives': [],
            'children': []
        },
        'Inventory': {
            'primary': 'InventoryNet',
            'alternatives': ['Inventory'],
            'children': []
        },
        'Other Current Assets': {
            'primary': 'OtherAssetsCurrent',
            'alternatives': ['OtherCurrentAssets'],
            'children': ['PrepaidExpensesCurrent']
        },
        'Total Current Assets': {
            'primary': 'AssetsCurrent',
            'alternatives': ['CurrentAssets'],
            'children': []
        },
        
        # Non-current Assets
        'Property, Plant and Equipment, Net': {
            'primary': 'PropertyPlantAndEquipmentNet',
            'alternatives': ['PropertyAndEquipmentNet'],
            'children': []
        },
        'Operating Lease Right-of-Use Assets': {
            'primary': 'OperatingLeaseRightOfUseAssets',
            'alternatives': [],
            'children': []
        },
        'Equity Investments': {
            'primary': 'EquityInvestments',
            'alternatives': [],
            'children': []
        },
        'Goodwill': {
            'primary': 'Goodwill',
            'alternatives': [],
            'children': []
        },
        'Intangible Assets, Net': {
            'primary': 'IntangibleAssetsNetExcludingGoodwill',
            'alternatives': ['IntangibleAssetsNet'],
            'children': []
        },
        'Other Long-term Assets': {
            'primary': 'OtherAssetsNoncurrent',
            'alternatives': ['OtherNoncurrentAssets', 'OtherLongTermAssets'],
            'children': ['DeferredIncomeTaxesNoncurrent']
        },
        'Total Assets': {
            'primary': 'Assets',
            'alternatives': [],
            'children': []
        }
    }
    
    # Liabilities section
    liabilities = {
        # Current Liabilities
        'Accounts Payable': {
            'primary': 'AccountsPayableCurrent',
            'alternatives': ['AccountsPayable'],
            'children': []
        },
        'Current Portion of Long-term Debt': {
            'primary': 'LongTermDebtCurrent',
            'alternatives': ['DebtCurrent'],
            'children': []
        },
        'Accrued Compensation': {
            'primary': 'AccruedCompensation',
            'alternatives': [],
            'children': []
        },
        'Income Taxes Payable': {
            'primary': 'IncomeTaxesPayableCurrent',
            'alternatives': ['IncomeTaxPayableCurrent'],
            'children': []
        },
        'Unearned Revenue (Current)': {
            'primary': 'DeferredRevenueCurrent',
            'alternatives': ['UnearnedRevenueCurrent', 'ContractLiabilitiesCurrent'],
            'children': []
        },
        'Other Current Liabilities': {
            'primary': 'OtherLiabilitiesCurrent',
            'alternatives': ['OtherCurrentLiabilities', 'OtherAccruedLiabilitiesCurrent'],
            'children': []
        },
        'Total Current Liabilities': {
            'primary': 'LiabilitiesCurrent',
            'alternatives': ['CurrentLiabilities'],
            'children': []
        },
        
        # Non-current Liabilities
        'Long-term Debt': {
            'primary': 'LongTermDebtNoncurrent',
            'alternatives': ['DebtNoncurrent', 'LongTermDebt'],
            'children': []
        },
        'Long-term Income Taxes': {
            'primary': 'IncomeTaxesPayableNoncurrent',
            'alternatives': ['IncomeTaxPayableNoncurrent', 'LongTermIncomeTaxPayable'],
            'children': []
        },
        'Long-term Unearned Revenue': {
            'primary': 'DeferredRevenueNoncurrent',
            'alternatives': ['UnearnedRevenueNoncurrent', 'ContractLiabilitiesNoncurrent'],
            'children': []
        },
        'Operating Lease Liabilities (Non-current)': {
            'primary': 'OperatingLeaseLiabilityNoncurrent',
            'alternatives': [],
            'children': []
        },
        'Other Long-term Liabilities': {
            'primary': 'OtherLiabilitiesNoncurrent',
            'alternatives': ['OtherNoncurrentLiabilities', 'OtherLongTermLiabilities'],
            'children': []
        },
        'Total Liabilities': {
            'primary': 'Liabilities',
            'alternatives': [],
            'children': []
        }
    }
    
    # Equity section
    equity = {
        'Common Stock and Paid-in Capital': {
            'primary': 'CommonStockAndAdditionalPaidInCapital',
            'alternatives': [],
            'children': ['CommonStockValue', 'AdditionalPaidInCapital']
        },
        'Retained Earnings': {
            'primary': 'RetainedEarningsAccumulatedDeficit',
            'alternatives': ['RetainedEarnings'],
            'children': []
        },
        'Accumulated Other Comprehensive Income': {
            'primary': 'AccumulatedOtherComprehensiveIncomeLoss',
            'alternatives': [],
            'children': []
        },
        'Total Stockholders Equity': {
            'primary': 'StockholdersEquity',
            'alternatives': ['TotalEquity'],
            'children': []
        },
        'Total Liabilities and Equity': {
            'primary': 'LiabilitiesAndStockholdersEquity',
            'alternatives': ['LiabilitiesAndEquity'],
            'children': []
        }
    }
    
    # Helper function to find the best matching column
    def find_best_match(account_info, available_cols):
        # Check primary tag
        if account_info['primary'] in available_cols:
            return account_info['primary']
        
        # Check alternatives
        for alt in account_info['alternatives']:
            if alt in available_cols:
                return alt
                
        # Handle partial matches for more flexible matching
        for col in available_cols:
            if account_info['primary'] in col:
                return col
                
        for alt in account_info['alternatives']:
            for col in available_cols:
                if alt in col:
                    return col
        
        return None
    
    # Process each section
    def process_section(section_name, section_mapping):
        for account_name, account_info in section_mapping.items():
            # Try to find the best match
            best_match = find_best_match(account_info, available_columns)
            
            if best_match:
                # Direct match found
                balance_sheet[f"{section_name} - {account_name}"] = df[best_match]
            elif account_info['children']:
                # Try to sum children if parent not found
                available_children = []
                for child in account_info['children']:
                    if child in available_columns:
                        available_children.append(child)
                    else:
                        # Look for variations
                        for col in available_columns:
                            if child in col:
                                available_children.append(col)
                
                if available_children:
                    balance_sheet[f"{section_name} - {account_name}"] = df[available_children].sum(axis=1)
                else:
                    # No matches found
                    balance_sheet[f"{section_name} - {account_name}"] = pd.Series(np.nan, index=df.index)
            else:
                # No matches found
                balance_sheet[f"{section_name} - {account_name}"] = pd.Series(np.nan, index=df.index)
    
    # Process each section
    process_section("Assets", assets)
    process_section("Liabilities", liabilities)
    process_section("Equity", equity)
    
    # Add validation checks - this will be updated in the add_total_liabilities_and_equity function
    
    # Check if current assets and liabilities match their components
    if 'Assets - Total Current Assets' in balance_sheet.columns:
        current_asset_cols = [col for col in balance_sheet.columns if col.startswith('Assets - ') and 'Current' in col and 'Total' not in col]
        if current_asset_cols:
            balance_sheet['Validation - Current Assets Difference'] = balance_sheet['Assets - Total Current Assets'] - balance_sheet[current_asset_cols].sum(axis=1)
    
    if 'Liabilities - Total Current Liabilities' in balance_sheet.columns:
        current_liab_cols = [col for col in balance_sheet.columns if col.startswith('Liabilities - ') and 'Current' in col and 'Total' not in col]
        if current_liab_cols:
            balance_sheet['Validation - Current Liabilities Difference'] = balance_sheet['Liabilities - Total Current Liabilities'] - balance_sheet[current_liab_cols].sum(axis=1)
    
    return balance_sheet

# Function to display a human-readable balance sheet
def display_balance_sheet(balance_sheet, in_billions=True):
    """
    Display the balance sheet in a clean, readable format.
    
    Args:
        balance_sheet: DataFrame with the reconstructed balance sheet
        in_billions: If True, display values in billions; otherwise in millions
    
    Returns:
        DataFrame with the formatted balance sheet
    """
    # Make a copy to avoid modifying the original
    formatted_bs = balance_sheet.copy()
    
    # Identify numeric columns
    numeric_cols = [col for col in formatted_bs.columns if any(col.startswith(prefix) for prefix in ['Assets', 'Liabilities', 'Equity', 'Validation'])]
    
    # Convert to billions or millions
    divisor = 1_000_000_000 if in_billions else 1_000_000
    suffix = 'B' if in_billions else 'M'
    
    for col in numeric_cols:
        formatted_bs[col] = formatted_bs[col] / divisor
        
    # Format the date column if it exists
    if 'end' in formatted_bs.columns:
        try:
            formatted_bs['end'] = pd.to_datetime(formatted_bs['end']).dt.strftime('%Y-%m-%d')
        except:
            pass
    
    # Create sections for display
    asset_cols = [col for col in formatted_bs.columns if col.startswith('Assets')]
    liability_cols = [col for col in formatted_bs.columns if col.startswith('Liabilities')]
    equity_cols = [col for col in formatted_bs.columns if col.startswith('Equity')]
    validation_cols = [col for col in formatted_bs.columns if col.startswith('Validation')]
    
    # Columns to display
    display_cols = ['end'] if 'end' in formatted_bs.columns else []
    display_cols.extend(asset_cols + liability_cols + equity_cols + validation_cols)
    
    return formatted_bs[display_cols]

# Function to add Total Liabilities and Equity if it doesn't exist
def add_total_liabilities_and_equity(balance_sheet):
    """
    Adds Total Liabilities and Equity column if it doesn't exist already.
    Ensures it equals Total Assets for validation.
    
    Args:
        balance_sheet: DataFrame with the reconstructed balance sheet
    
    Returns:
        DataFrame with Total Liabilities and Equity added
    """
    if 'Equity - Total Liabilities and Equity' not in balance_sheet.columns:
        if 'Liabilities - Total Liabilities' in balance_sheet.columns and 'Equity - Total Stockholders\' Equity' in balance_sheet.columns:
            balance_sheet['Equity - Total Liabilities and Equity'] = (
                balance_sheet['Liabilities - Total Liabilities'] + 
                balance_sheet['Equity - Total Stockholders\' Equity']
            )
    
    # Add validation check to ensure Total Assets = Total Liabilities and Equity
    if 'Assets - Total Assets' in balance_sheet.columns and 'Equity - Total Liabilities and Equity' in balance_sheet.columns:
        balance_sheet['Validation - A = L+E Difference'] = (
            balance_sheet['Assets - Total Assets'] - 
            balance_sheet['Equity - Total Liabilities and Equity']
        )
    
    return balance_sheet

# Function to remove columns that only contain NaN values
def remove_nan_only_columns(df):
    """
    Removes columns that contain only NaN values.
    
    Args:
        df: DataFrame to clean
    
    Returns:
        DataFrame with NaN-only columns removed
    """
    # Find columns where all values are NaN
    nan_cols = df.columns[df.isna().all()].tolist()
    
    # Drop those columns
    return df.drop(columns=nan_cols)
def add_missing_balance_sheet_totals(balance_sheet):
    """
    Adds missing total columns according to accounting relationships:
    - If all accounts exist, do nothing
    - If Total Liabilities is missing, compute it as (Total Liabilities and Equity - Total Stockholders Equity)
    - If Total Stockholders Equity is missing, compute it as (Total Liabilities and Equity - Total Liabilities)
    - If Total Liabilities and Equity is missing, set it equal to Total Assets
    
    Args:
        balance_sheet: DataFrame with the reconstructed balance sheet
    
    Returns:
        DataFrame with missing totals computed where possible
    """
    # Make a copy to avoid modifying the original
    result = balance_sheet.copy()
    
    # Check for the existence of key columns
    has_total_assets = 'Assets - Total Assets' in result.columns
    has_total_liabilities = 'Liabilities - Total Liabilities' in result.columns
    has_total_equity = 'Equity - Total Stockholders Equity' in result.columns
    has_total_liab_equity = 'Equity - Total Liabilities and Equity' in result.columns
    
    # Process at column level first (if entire column is missing)
    # Case 1: Compute missing Total Liabilities
    if not has_total_liabilities and has_total_liab_equity and has_total_equity:
        result['Liabilities - Total Liabilities'] = (
            result['Equity - Total Liabilities and Equity'] - 
            result['Equity - Total Stockholders Equity']
        )
        print("Computed missing Total Liabilities column")
        has_total_liabilities = True
    
    # Case 2: Compute missing Total Stockholders Equity
    if not has_total_equity and has_total_liab_equity and has_total_liabilities:
        result['Equity - Total Stockholders Equity'] = (
            result['Equity - Total Liabilities and Equity'] - 
            result['Liabilities - Total Liabilities']
        )
        print("Computed missing Total Stockholders Equity column")
        has_total_equity = True
    
    # Case 3: Compute missing Total Liabilities and Equity
    if not has_total_liab_equity:
        if has_total_assets:
            # Set Total Liabilities and Equity = Total Assets (accounting equality)
            result['Equity - Total Liabilities and Equity'] = result['Assets - Total Assets']
            print("Set Total Liabilities and Equity equal to Total Assets")
            has_total_liab_equity = True
        elif has_total_liabilities and has_total_equity:
            # Compute Total Liabilities and Equity as sum of components
            result['Equity - Total Liabilities and Equity'] = (
                result['Liabilities - Total Liabilities'] + 
                result['Equity - Total Stockholders Equity']
            )
            print("Computed Total Liabilities and Equity from its components")
            has_total_liab_equity = True
    
    # Now process row by row to handle NaN values in specific cells
    if has_total_liabilities and has_total_equity and has_total_liab_equity:
        for idx, row in result.iterrows():
            # Case 1: Compute missing Total Liabilities in specific rows
            if (pd.notna(row['Equity - Total Liabilities and Equity']) and 
                pd.notna(row['Equity - Total Stockholders Equity']) and
                pd.isna(row['Liabilities - Total Liabilities'])):
                
                result.at[idx, 'Liabilities - Total Liabilities'] = (
                    row['Equity - Total Liabilities and Equity'] - 
                    row['Equity - Total Stockholders Equity']
                )
                print(f"Row {idx}: Computed missing Total Liabilities")
            
            # Case 2: Compute missing Total Stockholders Equity in specific rows
            if (pd.notna(row['Equity - Total Liabilities and Equity']) and 
                pd.notna(row['Liabilities - Total Liabilities']) and
                pd.isna(row['Equity - Total Stockholders Equity'])):
                
                result.at[idx, 'Equity - Total Stockholders Equity'] = (
                    row['Equity - Total Liabilities and Equity'] - 
                    row['Liabilities - Total Liabilities']
                )
                print(f"Row {idx}: Computed missing Total Stockholders Equity")
            
            # Case 3: Compute missing Total Liabilities and Equity in specific rows
            if pd.isna(row['Equity - Total Liabilities and Equity']):
                if pd.notna(row['Assets - Total Assets']):
                    # Set Total Liabilities and Equity = Total Assets (accounting equality)
                    result.at[idx, 'Equity - Total Liabilities and Equity'] = row['Assets - Total Assets']
                    print(f"Row {idx}: Set Total Liabilities and Equity equal to Total Assets")
                elif (pd.notna(row['Liabilities - Total Liabilities']) and 
                      pd.notna(row['Equity - Total Stockholders Equity'])):
                    # Compute Total Liabilities and Equity as sum of components
                    result.at[idx, 'Equity - Total Liabilities and Equity'] = (
                        row['Liabilities - Total Liabilities'] + 
                        row['Equity - Total Stockholders Equity']
                    )
                    print(f"Row {idx}: Computed Total Liabilities and Equity from its components")
    
    # Add validation columns
    if has_total_assets and has_total_liab_equity:
        result['Validation - A = L+E Difference'] = (
            result['Assets - Total Assets'] - 
            result['Equity - Total Liabilities and Equity']
        )
    
    if has_total_liabilities and has_total_equity and has_total_liab_equity:
        result['Validation - Components = Total L+E Difference'] = (
            result['Liabilities - Total Liabilities'] + 
            result['Equity - Total Stockholders Equity'] - 
            result['Equity - Total Liabilities and Equity']
        )
    
    return result

def analyze_balance_sheet(df):
    """
    Analyze company balance sheet from XBRL data.
    
    Args:
        df: DataFrame containing XBRL-tagged financial data
    
    Returns:
        The reconstructed balance sheet
    """
    print("Reconstructing balance sheet from XBRL data...")
    balance_sheet = reconstruct_balance_sheet(df)
    
    # Add missing totals based on accounting relationships
    balance_sheet = add_missing_balance_sheet_totals(balance_sheet)
    
    # Remove columns with only NaN values
    balance_sheet = remove_nan_only_columns(balance_sheet)
    
    print("\nBalance Sheet Summary:")
    print(f"- Total columns: {len(balance_sheet.columns)}")
    print(f"- Data for {len(balance_sheet)} reporting periods")
    
    # Display the balance sheet
    print("\nFormatted Balance Sheet (in billions USD):")
    formatted_bs = display_balance_sheet(balance_sheet, in_billions=True)
    
    return balance_sheet, formatted_bs

# To use this code with your DataFrame:
# balance_sheet, formatted_bs = analyze_balance_sheet(consolidated_df)

# To use this code with your DataFrame:
balance_sheet, formatted_bs = analyze_balance_sheet(consolidated_df)

Reconstructing balance sheet from XBRL data...
Row 48: Computed missing Total Liabilities
Row 53: Computed missing Total Liabilities
Row 55: Computed missing Total Liabilities
Row 56: Computed missing Total Liabilities
Row 58: Computed missing Total Liabilities
Row 60: Computed missing Total Liabilities
Row 61: Computed missing Total Liabilities
Row 63: Computed missing Total Liabilities
Row 65: Computed missing Total Liabilities
Row 66: Computed missing Total Liabilities
Row 68: Computed missing Total Liabilities
Row 71: Computed missing Total Liabilities
Row 72: Computed missing Total Liabilities
Row 74: Computed missing Total Liabilities
Row 76: Computed missing Total Liabilities
Row 77: Computed missing Total Liabilities
Row 79: Computed missing Total Liabilities
Row 81: Computed missing Total Liabilities
Row 82: Computed missing Total Liabilities

Balance Sheet Summary:
- Total columns: 34
- Data for 26 reporting periods

Formatted Balance Sheet (in billions USD):


In [47]:
yearly = balance_sheet[balance_sheet['frame'].str.match(r'^CY\d{4}$')]
yearly.T

,48,52,57,58,62,63,67,68,72,73,74,78,79,82,83
filed,2019-04-26,2019-02-01,2020-01-31,2021-04-30,2021-02-03,2022-04-29,2022-02-04,2022-04-29,2022-10-28,2023-02-03,2024-05-01,2024-02-02,2024-05-01,2024-11-01,2025-02-07
company_name,"AMAZON.COM, INC.","AMAZON.COM, INC.","AMAZON.COM, INC.","AMAZON.COM, INC.","AMAZON.COM, INC.","AMAZON.COM, INC.","AMAZON.COM, INC.","AMAZON.COM, INC.","AMAZON.COM, INC.","AMAZON.COM, INC.","AMAZON.COM, INC.","AMAZON.COM, INC.","AMAZON.COM, INC.","AMAZON.COM, INC.","AMAZON.COM, INC."
end,2018-03-31,2018-12-31,2019-12-31,2020-03-31,2020-12-31,2021-03-31,2021-12-31,2022-03-31,2022-09-30,2022-12-31,2023-03-31,2023-12-31,2024-03-31,2024-09-30,2024-12-31
unit,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD
form,10-Q,10-K,10-K,10-Q,10-K,10-Q,10-K,10-Q,10-Q,10-K,10-Q,10-K,10-Q,10-Q,10-K
frame,CY2017,CY2018,CY2019,CY2019,CY2020,CY2020,CY2021,CY2021,CY2022,CY2022,CY2022,CY2023,CY2023,CY2024,CY2024
cik,1018724,1018724,1018724,1018724,1018724,1018724,1018724,1018724,1018724,1018724,1018724,1018724,1018724,1018724,1018724
Assets - Cash and Cash Equivalents,16676000000.0,31750000000.0,36092000000.0,27201000000.0,42122000000.0,33834000000.0,36220000000.0,36393000000.0,34947000000.0,53888000000.0,49343000000.0,73387000000.0,72852000000.0,75091000000.0,78779000000.0
Assets - Short-term Investments,8287000000.0,9500000000.0,18929000000.0,22091000000.0,42274000000.0,39436000000.0,59829000000.0,29992000000.0,23715000000.0,16138000000.0,15062000000.0,13393000000.0,12222000000.0,12960000000.0,22423000000.0
"Assets - Accounts Receivable, Net (Current)",12026000000.0,16677000000.0,20816000000.0,17836000000.0,24542000000.0,24289000000.0,32891000000.0,32504000000.0,36154000000.0,42360000000.0,37646000000.0,52253000000.0,47768000000.0,51638000000.0,55451000000.0


# Balance Sheet after removing the columns that were not usefull for our workflow

In [48]:
# Keep rows with exactly Q1, Q2, Q3, Q4 or just the year (like CY2019)
balance_sheet = balance_sheet[~balance_sheet['frame'].str.match(r'CY\d{4}Q[1-4].+')]
balance_sheet = balance_sheet.sort_values(by='end')
balance_sheet.T

,48,52,53,55,56,57,58,60,61,62,...,72,73,74,76,77,78,79,81,82,83
filed,2019-04-26,2019-02-01,2019-04-26,2019-07-26,2019-10-25,2020-01-31,2021-04-30,2021-07-30,2020-10-30,2021-02-03,...,2022-10-28,2023-02-03,2024-05-01,2023-08-04,2023-10-27,2024-02-02,2024-05-01,2024-08-02,2024-11-01,2025-02-07
company_name,"AMAZON.COM, INC.","AMAZON.COM, INC.","AMAZON.COM, INC.","AMAZON.COM, INC.","AMAZON.COM, INC.","AMAZON.COM, INC.","AMAZON.COM, INC.","AMAZON.COM, INC.","AMAZON.COM, INC.","AMAZON.COM, INC.",...,"AMAZON.COM, INC.","AMAZON.COM, INC.","AMAZON.COM, INC.","AMAZON.COM, INC.","AMAZON.COM, INC.","AMAZON.COM, INC.","AMAZON.COM, INC.","AMAZON.COM, INC.","AMAZON.COM, INC.","AMAZON.COM, INC."
end,2018-03-31,2018-12-31,2019-03-31,2019-06-30,2019-09-30,2019-12-31,2020-03-31,2020-06-30,2020-09-30,2020-12-31,...,2022-09-30,2022-12-31,2023-03-31,2023-06-30,2023-09-30,2023-12-31,2024-03-31,2024-06-30,2024-09-30,2024-12-31
unit,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD,...,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD
form,10-Q,10-K,10-Q,10-Q,10-Q,10-K,10-Q,10-Q,10-Q,10-K,...,10-Q,10-K,10-Q,10-Q,10-Q,10-K,10-Q,10-Q,10-Q,10-K
frame,CY2017,CY2018,CY2019Q1,CY2019Q2,CY2019Q3,CY2019,CY2019,CY2020Q2,CY2020Q3,CY2020,...,CY2022,CY2022,CY2022,CY2023Q2,CY2023Q3,CY2023,CY2023,CY2024Q2,CY2024,CY2024
cik,1018724,1018724,1018724,1018724,1018724,1018724,1018724,1018724,1018724,1018724,...,1018724,1018724,1018724,1018724,1018724,1018724,1018724,1018724,1018724,1018724
Assets - Cash and Cash Equivalents,16676000000.0,31750000000.0,23115000000.0,22616000000.0,23255000000.0,36092000000.0,27201000000.0,37466000000.0,29930000000.0,42122000000.0,...,34947000000.0,53888000000.0,49343000000.0,49529000000.0,49605000000.0,73387000000.0,72852000000.0,71178000000.0,75091000000.0,78779000000.0
Assets - Short-term Investments,8287000000.0,9500000000.0,13905000000.0,18847000000.0,20146000000.0,18929000000.0,22091000000.0,33925000000.0,38472000000.0,42274000000.0,...,23715000000.0,16138000000.0,15062000000.0,14441000000.0,14564000000.0,13393000000.0,12222000000.0,17914000000.0,12960000000.0,22423000000.0
"Assets - Accounts Receivable, Net (Current)",12026000000.0,16677000000.0,15979000000.0,16747000000.0,16887000000.0,20816000000.0,17836000000.0,19918000000.0,20832000000.0,24542000000.0,...,36154000000.0,42360000000.0,37646000000.0,39925000000.0,43420000000.0,52253000000.0,47768000000.0,50106000000.0,51638000000.0,55451000000.0


In [49]:
balance_sheet.columns

Index(['filed', 'company_name', 'end', 'unit', 'form', 'frame', 'cik',
       'Assets - Cash and Cash Equivalents', 'Assets - Short-term Investments',
       'Assets - Accounts Receivable, Net (Current)', 'Assets - Inventory',
       'Assets - Other Current Assets', 'Assets - Total Current Assets',
       'Assets - Property, Plant and Equipment, Net', 'Assets - Goodwill',
       'Assets - Intangible Assets, Net', 'Assets - Other Long-term Assets',
       'Assets - Total Assets', 'Liabilities - Accounts Payable',
       'Liabilities - Current Portion of Long-term Debt',
       'Liabilities - Total Current Liabilities',
       'Liabilities - Long-term Debt',
       'Liabilities - Operating Lease Liabilities (Non-current)',
       'Liabilities - Other Long-term Liabilities',
       'Liabilities - Total Liabilities',
       'Equity - Common Stock and Paid-in Capital',
       'Equity - Retained Earnings',
       'Equity - Accumulated Other Comprehensive Income',
       'Equity - Total Stock

### Income statement

In [50]:
import pandas as pd
import numpy as np

def reconstruct_income_statement(df):
    """
    Reconstructs income statement from XBRL-tagged data, specifically tailored for IT companies like Microsoft.
    
    Args:
        df: DataFrame containing XBRL-tagged financial data
        
    Returns:
        DataFrame with reconstructed income statement for each period
    """
    # Create a new DataFrame to store the reconstructed income statement
    income_statement = pd.DataFrame(index=df.index)
    
    # Copy identifying columns
    id_columns = ['filed', 'company_name', 'end', 'unit', 'form', 'frame', 'cik']
    for col in id_columns:
        if col in df.columns:
            income_statement[col] = df[col]
    
    # Get all available columns for matching
    available_columns = set(df.columns)
    
    # Define mappings for income statement accounts based on XBRL structure
    
    # Revenue section
    revenue = {
        'Total Revenue': {
            'primary': 'RevenueFromContractWithCustomerExcludingAssessedTax',
            'alternatives': ['SalesRevenueNet', 'Revenue', 'Revenues'],
            'children': ['ProductRevenue', 'ServiceRevenue', 'CloudComputingArrangementRevenue', 'SubscriptionServiceRevenue']
        },
        'Product Revenue': {
            'primary': 'ProductRevenue',
            'alternatives': ['SalesOfProductsNet'],
            'children': []
        },
        'Service Revenue': {
            'primary': 'ServiceRevenue',
            'alternatives': ['SalesOfServicesNet'],
            'children': []
        },
        'Cloud Revenue': {
            'primary': 'CloudComputingArrangementRevenue',
            'alternatives': ['HostingServiceRevenue', 'SubscriptionServiceRevenue'],
            'children': []
        }
    }
    
    # Cost of Revenue section
    cost_of_revenue = {
        'Total Cost of Revenue': {
            'primary': 'CostOfRevenue',
            'alternatives': ['CostOfGoodsSold', 'CostOfSales', 'CostOfGoodsAndServicesSold'],
            'children': ['CostOfProductRevenue', 'CostOfServiceRevenue']
        },
        'Cost of Product Revenue': {
            'primary': 'CostOfProductRevenue',
            'alternatives': ['CostOfGoodsSoldProduct'],
            'children': []
        },
        'Cost of Service Revenue': {
            'primary': 'CostOfServiceRevenue',
            'alternatives': ['CostOfServicesSold'],
            'children': []
        }
    }
    
    # Gross Profit
    gross_profit = {
        'Gross Profit': {
            'primary': 'GrossProfit',
            'alternatives': [],
            'children': []
        }
    }
    
    # Operating Expenses section
    operating_expenses = {
        'Research and Development': {
            'primary': 'ResearchAndDevelopmentExpense',
            'alternatives': ['ResearchDevelopmentExpense'],
            'children': []
        },
        'Sales and Marketing': {
            'primary': 'SalesAndMarketingExpense',
            'alternatives': ['MarketingExpense', 'SellingExpense', 'SellingAndMarketingExpense'],
            'children': []
        },
        'General and Administrative': {
            'primary': 'GeneralAndAdministrativeExpense',
            'alternatives': ['AdminExpense'],
            'children': []
        },
        'Selling, General and Administrative': {
            'primary': 'SellingGeneralAndAdministrativeExpenses',
            'alternatives': ['GeneralAndAdministrativeExpense'],
            'children': ['SalesAndMarketingExpense', 'GeneralAndAdministrativeExpense']
        },
        'Amortization of Intangible Assets': {
            'primary': 'AmortizationOfIntangibleAssets',
            'alternatives': ['AmortizationOfIntangibleAssetsExcludingGoodwill'],
            'children': []
        },
        'Restructuring Charges': {
            'primary': 'RestructuringCharges',
            'alternatives': ['RestructuringCosts'],
            'children': []
        },
        'Impairment of Goodwill': {
            'primary': 'ImpairmentOfGoodwill',
            'alternatives': [],
            'children': []
        },
        'Impairment of Intangible Assets': {
            'primary': 'ImpairmentOfIntangibleAssets',
            'alternatives': [],
            'children': []
        },
        'Total Operating Expenses': {
            'primary': 'OperatingExpenses',
            'alternatives': ['TotalOperatingExpenses'],
            'children': []
        }
    }
    
    # Operating Income
    operating_income = {
        'Operating Income': {
            'primary': 'IncomeLossFromOperations',
            'alternatives': ['OperatingIncomeLoss', 'IncomeFromOperations'],
            'children': []
        }
    }
    
    # Other Income/Expense section
    other_income_expense = {
        'Interest Income': {
            'primary': 'InterestIncome',
            'alternatives': ['InvestmentIncomeInterest'],
            'children': []
        },
        'Interest Expense': {
            'primary': 'InterestExpense',
            'alternatives': ['InterestCost'],
            'children': []
        },
        'Gain/Loss on Investments': {
            'primary': 'GainLossOnInvestments',
            'alternatives': ['InvestmentIncomeNetGainLossFromSaleOfInvestment'],
            'children': []
        },
        'Foreign Currency Transaction Gain/Loss': {
            'primary': 'ForeignCurrencyTransactionGainLoss',
            'alternatives': [],
            'children': []
        },
        'Other Income/Expense, Net': {
            'primary': 'OtherIncomeExpenseNet',
            'alternatives': ['NonoperatingIncomeExpense'],
            'children': []
        }
    }
    
    # Income before taxes
    income_before_tax = {
        'Income Before Income Taxes': {
            'primary': 'IncomeLossBeforeIncomeTax',
            'alternatives': ['IncomeBeforeTax', 'EarningsBeforeIncomeTax'],
            'children': []
        }
    }
    
    # Income tax provision
    income_tax = {
        'Income Tax Expense/Benefit': {
            'primary': 'IncomeTaxExpenseBenefit',
            'alternatives': ['IncomeTaxesPaidCurrent', 'IncomeTaxesPaid'],
            'children': []
        }
    }
    
    # Net Income
    net_income = {
        'Net Income': {
            'primary': 'ProfitLoss',
            'alternatives': ['NetIncomeLoss'],
            'children': []
        },
        'Net Income Attributable to Noncontrolling Interest': {
            'primary': 'NetIncomeLossAttributableToNoncontrollingInterest',
            'alternatives': [],
            'children': []
        },
        'Net Income Attributable to Company': {
            'primary': 'NetIncomeLossAttributableToParent',
            'alternatives': ['NetIncomeLossAvailableToCommonStockholdersBasic'],
            'children': []
        }
    }
    
    # Earnings Per Share
    eps = {
        'Basic EPS': {
            'primary': 'EarningsPerShareBasic',
            'alternatives': [],
            'children': []
        },
        'Diluted EPS': {
            'primary': 'EarningsPerShareDiluted',
            'alternatives': [],
            'children': []
        }
    }
    
    # Helper function to find the best matching column
    def find_best_match(account_info, available_cols):
        # Check primary tag
        if account_info['primary'] in available_cols:
            return account_info['primary']
        
        # Check alternatives
        for alt in account_info['alternatives']:
            if alt in available_cols:
                return alt
                
        # Handle partial matches for more flexible matching
        primary_matches = []
        for col in available_cols:
            if account_info['primary'] in col:
                primary_matches.append(col)
        
        # If we found exactly one partial match for primary, return it
        if len(primary_matches) == 1:
            return primary_matches[0]
                
        # If we found multiple partial matches, try to find the best one
        if primary_matches:
            # For certain important accounts, use a more specific matching strategy
            if account_info['primary'] in ['ProfitLoss', 'RevenueFromContractWithCustomer', 'CostOfRevenue']:
                for match in primary_matches:
                    if account_info['primary'] in match and not any(x in match.lower() for x in ['segment', 'division', 'product']):
                        return match
            
            return primary_matches[0]  # Default to first one
        
        # Check for partial matches in alternatives
        for alt in account_info['alternatives']:
            for col in available_cols:
                if alt in col:
                    return col
        
        return None
    
    # Process each section
    def process_section(section_name, section_mapping):
        for account_name, account_info in section_mapping.items():
            # Try to find the best match
            best_match = find_best_match(account_info, available_columns)
            
            if best_match:
                # Direct match found
                income_statement[f"{section_name} - {account_name}"] = df[best_match]
            elif account_info['children']:
                # Try to sum children if parent not found
                available_children = []
                for child in account_info['children']:
                    if child in available_columns:
                        available_children.append(child)
                    else:
                        # Look for variations
                        for col in available_columns:
                            if child in col:
                                available_children.append(col)
                
                if available_children:
                    income_statement[f"{section_name} - {account_name}"] = df[available_children].sum(axis=1)
                else:
                    # No matches found
                    income_statement[f"{section_name} - {account_name}"] = pd.Series(np.nan, index=df.index)
            else:
                # No matches found
                income_statement[f"{section_name} - {account_name}"] = pd.Series(np.nan, index=df.index)
    
    # Process each section
    process_section("Revenue", revenue)
    process_section("Cost", cost_of_revenue)
    process_section("Gross", gross_profit)
    process_section("Operating Expenses", operating_expenses)
    process_section("Operating", operating_income)
    process_section("Other", other_income_expense)
    process_section("Income Before Tax", income_before_tax)
    process_section("Tax", income_tax)
    process_section("Net Income", net_income)
    process_section("EPS", eps)
    
    # Calculate missing values and add validation checks
    
    # If Gross Profit is missing, calculate it
    if ('Gross - Gross Profit' not in income_statement.columns or income_statement['Gross - Gross Profit'].isna().all()) and \
       'Revenue - Total Revenue' in income_statement.columns and 'Cost - Total Cost of Revenue' in income_statement.columns:
        income_statement['Gross - Gross Profit (calculated)'] = income_statement['Revenue - Total Revenue'] - income_statement['Cost - Total Cost of Revenue']
    
    # If Operating Income is missing, calculate it
    if ('Operating - Operating Income' not in income_statement.columns or income_statement['Operating - Operating Income'].isna().all()) and \
       'Gross - Gross Profit' in income_statement.columns and 'Operating Expenses - Total Operating Expenses' in income_statement.columns:
        income_statement['Operating - Operating Income (calculated)'] = income_statement['Gross - Gross Profit'] - income_statement['Operating Expenses - Total Operating Expenses']
    elif ('Operating - Operating Income' not in income_statement.columns or income_statement['Operating - Operating Income'].isna().all()) and \
         'Gross - Gross Profit (calculated)' in income_statement.columns and 'Operating Expenses - Total Operating Expenses' in income_statement.columns:
        income_statement['Operating - Operating Income (calculated)'] = income_statement['Gross - Gross Profit (calculated)'] - income_statement['Operating Expenses - Total Operating Expenses']
    
    # If Income Before Tax is missing, calculate it
    if ('Income Before Tax - Income Before Income Taxes' not in income_statement.columns or income_statement['Income Before Tax - Income Before Income Taxes'].isna().all()):
        # Try to calculate from Operating Income and Other Income/Expense
        if 'Operating - Operating Income' in income_statement.columns and 'Other - Other Income/Expense, Net' in income_statement.columns:
            income_statement['Income Before Tax - Income Before Income Taxes (calculated)'] = income_statement['Operating - Operating Income'] + income_statement['Other - Other Income/Expense, Net']
        elif 'Operating - Operating Income (calculated)' in income_statement.columns and 'Other - Other Income/Expense, Net' in income_statement.columns:
            income_statement['Income Before Tax - Income Before Income Taxes (calculated)'] = income_statement['Operating - Operating Income (calculated)'] + income_statement['Other - Other Income/Expense, Net']
    
    # If Net Income is missing, calculate it
    if ('Net Income - Net Income' not in income_statement.columns or income_statement['Net Income - Net Income'].isna().all()):
        # Try to calculate from Income Before Tax and Income Tax Expense
        if 'Income Before Tax - Income Before Income Taxes' in income_statement.columns and 'Tax - Income Tax Expense/Benefit' in income_statement.columns:
            income_statement['Net Income - Net Income (calculated)'] = income_statement['Income Before Tax - Income Before Income Taxes'] - income_statement['Tax - Income Tax Expense/Benefit']
        elif 'Income Before Tax - Income Before Income Taxes (calculated)' in income_statement.columns and 'Tax - Income Tax Expense/Benefit' in income_statement.columns:
            income_statement['Net Income - Net Income (calculated)'] = income_statement['Income Before Tax - Income Before Income Taxes (calculated)'] - income_statement['Tax - Income Tax Expense/Benefit']
    
    # Add validation checks for key relationships
    # Validation: Revenue - COGS = Gross Profit
    if 'Revenue - Total Revenue' in income_statement.columns and 'Cost - Total Cost of Revenue' in income_statement.columns and 'Gross - Gross Profit' in income_statement.columns:
        income_statement['Validation - Gross Profit Difference'] = income_statement['Gross - Gross Profit'] - (income_statement['Revenue - Total Revenue'] - income_statement['Cost - Total Cost of Revenue'])
    
    # Validation: Gross Profit - Operating Expenses = Operating Income
    if 'Gross - Gross Profit' in income_statement.columns and 'Operating Expenses - Total Operating Expenses' in income_statement.columns and 'Operating - Operating Income' in income_statement.columns:
        income_statement['Validation - Operating Income Difference'] = income_statement['Operating - Operating Income'] - (income_statement['Gross - Gross Profit'] - income_statement['Operating Expenses - Total Operating Expenses'])
    
    # Validation: Income Before Tax - Income Tax = Net Income
    if 'Income Before Tax - Income Before Income Taxes' in income_statement.columns and 'Tax - Income Tax Expense/Benefit' in income_statement.columns and 'Net Income - Net Income' in income_statement.columns:
        income_statement['Validation - Net Income Difference'] = income_statement['Net Income - Net Income'] - (income_statement['Income Before Tax - Income Before Income Taxes'] - income_statement['Tax - Income Tax Expense/Benefit'])
    
    return income_statement

# Function to display a human-readable income statement
def display_income_statement(income_statement, in_millions=True):
    """
    Display the income statement in a clean, readable format.
    
    Args:
        income_statement: DataFrame with the reconstructed income statement
        in_millions: If True, display values in millions; otherwise in thousands
    
    Returns:
        DataFrame with the formatted income statement
    """
    # Make a copy to avoid modifying the original
    formatted_is = income_statement.copy()
    
    # Identify numeric columns
    numeric_cols = [col for col in formatted_is.columns if any(col.startswith(prefix) for prefix in ['Revenue', 'Cost', 'Gross', 'Operating', 'Other', 'Income Before Tax', 'Tax', 'Net Income', 'EPS', 'Validation'])]
    
    # Convert to millions or thousands
    divisor = 1_000_000 if in_millions else 1_000
    suffix = 'M' if in_millions else 'K'
    
    for col in numeric_cols:
        if 'EPS' not in col:  # Don't scale EPS values
            formatted_is[col] = formatted_is[col] / divisor
        
    # Format the date column if it exists
    if 'end' in formatted_is.columns:
        try:
            formatted_is['end'] = pd.to_datetime(formatted_is['end']).dt.strftime('%Y-%m-%d')
        except:
            pass
    
    # Create sections for display in proper income statement order
    display_sections = [
        'Revenue',
        'Cost',
        'Gross',
        'Operating Expenses',
        'Operating',
        'Other',
        'Income Before Tax',
        'Tax',
        'Net Income',
        'EPS'
    ]
    
    # Columns to display
    display_cols = ['end'] if 'end' in formatted_is.columns else []
    for section in display_sections:
        section_cols = [col for col in formatted_is.columns if col.startswith(f"{section} - ")]
        display_cols.extend(section_cols)
    
    # Add validation columns at the end if they exist
    validation_cols = [col for col in formatted_is.columns if col.startswith('Validation - ')]
    display_cols.extend(validation_cols)
    
    return formatted_is[display_cols]

# Function to remove columns that only contain NaN values
def remove_nan_only_columns(df):
    """
    Removes columns that contain only NaN values.
    
    Args:
        df: DataFrame to clean
    
    Returns:
        DataFrame with NaN-only columns removed
    """
    # Find columns where all values are NaN
    nan_cols = df.columns[df.isna().all()].tolist()
    
    # Drop those columns
    return df.drop(columns=nan_cols)

# Example usage
def analyze_microsoft_income_statement(df):
    """
    Analyze Microsoft's income statement from XBRL data.
    
    Args:
        df: DataFrame containing Microsoft's XBRL-tagged financial data
    
    Returns:
        The reconstructed income statement
    """
    print("Reconstructing Microsoft's income statement from XBRL data...")
    income_statement = reconstruct_income_statement(df)
    
    # Remove columns with only NaN values
    income_statement = remove_nan_only_columns(income_statement)
    
    print("\nIncome Statement Summary:")
    print(f"- Total columns: {len(income_statement.columns)}")
    print(f"- Data for {len(income_statement)} reporting periods")
    
    # Display the income statement
    print("\nFormatted Income Statement (in millions USD):")
    formatted_is = display_income_statement(income_statement, in_millions=True)
    
    return income_statement, formatted_is

# To use this code with your DataFrame:
income_statement, formatted_is = analyze_microsoft_income_statement(consolidated_df)

Reconstructing Microsoft's income statement from XBRL data...

Income Statement Summary:
- Total columns: 24
- Data for 26 reporting periods

Formatted Income Statement (in millions USD):


In [51]:
consolidated_df.to_csv('./test.csv')

In [52]:
# Keep rows with exactly Q1, Q2, Q3, Q4 or just the year (like CY2019)
income_statement = income_statement[~income_statement['frame'].str.match(r'CY\d{4}Q[1-4].+')]
income_statement = income_statement.sort_values(by='end')
income_statement.T

,48,52,53,55,56,57,58,60,61,62,...,72,73,74,76,77,78,79,81,82,83
filed,2019-04-26,2019-02-01,2019-04-26,2019-07-26,2019-10-25,2020-01-31,2021-04-30,2021-07-30,2020-10-30,2021-02-03,...,2022-10-28,2023-02-03,2024-05-01,2023-08-04,2023-10-27,2024-02-02,2024-05-01,2024-08-02,2024-11-01,2025-02-07
company_name,"AMAZON.COM, INC.","AMAZON.COM, INC.","AMAZON.COM, INC.","AMAZON.COM, INC.","AMAZON.COM, INC.","AMAZON.COM, INC.","AMAZON.COM, INC.","AMAZON.COM, INC.","AMAZON.COM, INC.","AMAZON.COM, INC.",...,"AMAZON.COM, INC.","AMAZON.COM, INC.","AMAZON.COM, INC.","AMAZON.COM, INC.","AMAZON.COM, INC.","AMAZON.COM, INC.","AMAZON.COM, INC.","AMAZON.COM, INC.","AMAZON.COM, INC.","AMAZON.COM, INC."
end,2018-03-31,2018-12-31,2019-03-31,2019-06-30,2019-09-30,2019-12-31,2020-03-31,2020-06-30,2020-09-30,2020-12-31,...,2022-09-30,2022-12-31,2023-03-31,2023-06-30,2023-09-30,2023-12-31,2024-03-31,2024-06-30,2024-09-30,2024-12-31
unit,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD,...,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD
form,10-Q,10-K,10-Q,10-Q,10-Q,10-K,10-Q,10-Q,10-Q,10-K,...,10-Q,10-K,10-Q,10-Q,10-Q,10-K,10-Q,10-Q,10-Q,10-K
frame,CY2017,CY2018,CY2019Q1,CY2019Q2,CY2019Q3,CY2019,CY2019,CY2020Q2,CY2020Q3,CY2020,...,CY2022,CY2022,CY2022,CY2023Q2,CY2023Q3,CY2023,CY2023,CY2024Q2,CY2024,CY2024
cik,1018724,1018724,1018724,1018724,1018724,1018724,1018724,1018724,1018724,1018724,...,1018724,1018724,1018724,1018724,1018724,1018724,1018724,1018724,1018724,1018724
Revenue - Total Revenue,51042000000.0,232887000000.0,59700000000.0,63404000000.0,69981000000.0,280522000000.0,75452000000.0,88912000000.0,96145000000.0,386064000000.0,...,127101000000.0,513983000000.0,127358000000.0,134383000000.0,143083000000.0,574785000000.0,143313000000.0,147977000000.0,158877000000.0,637959000000.0
Cost - Total Cost of Revenue,30735000000.0,139156000000.0,33920000000.0,36337000000.0,41302000000.0,165536000000.0,44257000000.0,52660000000.0,57106000000.0,233307000000.0,...,70268000000.0,288831000000.0,67791000000.0,69373000000.0,75022000000.0,304739000000.0,72633000000.0,73785000000.0,80977000000.0,326288000000.0
Operating Expenses - Sales and Marketing,2699000000.0,13814000000.0,3664000000.0,4291000000.0,4752000000.0,18878000000.0,4828000000.0,4345000000.0,5434000000.0,22008000000.0,...,11014000000.0,42238000000.0,10172000000.0,10745000000.0,10551000000.0,44370000000.0,9662000000.0,10512000000.0,10609000000.0,43907000000.0


## CashFlow Statement

In [53]:
import pandas as pd
import numpy as np

def reconstruct_cash_flow_statement(df):
    """
    Reconstructs cash flow statement from XBRL-tagged data, specifically tailored for IT companies like Microsoft.
    
    Args:
        df: DataFrame containing XBRL-tagged financial data
        
    Returns:
        DataFrame with reconstructed cash flow statement for each period
    """
    # Create a new DataFrame to store the reconstructed cash flow statement
    cash_flow = pd.DataFrame(index=df.index)
    
    # Copy identifying columns
    id_columns = ['filed', 'company_name', 'end', 'unit', 'form', 'frame', 'cik']
    for col in id_columns:
        if col in df.columns:
            cash_flow[col] = df[col]
    
    # Get all available columns for matching
    available_columns = set(df.columns)
    
    # Define mappings for cash flow statement accounts based on XBRL structure
    
    # Operating Activities section
    operating_activities = {
        'Net Income': {
            'primary': 'ProfitLoss',
            'alternatives': ['NetIncomeLoss'],
            'children': []
        },
        'Depreciation and Amortization': {
            'primary': 'DepreciationAndAmortization',
            'alternatives': [],
            'children': ['Depreciation', 'AmortizationOfIntangibleAssets', 'AmortizationOfCapitalizedSoftwareCosts']
        },
        'Stock-based Compensation': {
            'primary': 'StockBasedCompensationExpense',
            'alternatives': ['ShareBasedCompensation'],
            'children': []
        },
        'Deferred Income Taxes': {
            'primary': 'DeferredIncomeTaxExpenseBenefit',
            'alternatives': ['IncomeTaxExpenseBenefitDeferredTax'],
            'children': []
        },
        'Impairment of Goodwill': {
            'primary': 'ImpairmentOfGoodwill',
            'alternatives': [],
            'children': []
        },
        'Impairment of Intangible Assets': {
            'primary': 'ImpairmentOfIntangibleAssets',
            'alternatives': [],
            'children': []
        },
        'Net Gains/Losses on Investments': {
            'primary': 'GainLossOnInvestments',
            'alternatives': ['InvestmentIncomeNetGainLossFromSaleOfInvestment'],
            'children': []
        },
        'Net Gains/Losses on Asset Sales': {
            'primary': 'GainLossOnSaleOfPropertyPlantAndEquipment',
            'alternatives': [],
            'children': []
        },
        'Changes in Operating Assets and Liabilities': {
            'primary': 'ChangeInOperatingAssetsAndLiabilities',
            'alternatives': [],
            'children': []
        },
        'Change in Accounts Receivable': {
            'primary': 'ChangeInAccountsReceivable',
            'alternatives': [],
            'children': []
        },
        'Change in Inventory': {
            'primary': 'ChangeInInventory',
            'alternatives': [],
            'children': []
        },
        'Change in Prepaid Expenses and Other Assets': {
            'primary': 'ChangeInPrepaidExpensesAndOtherAssets',
            'alternatives': [],
            'children': ['ChangeInPrepaidExpenses', 'ChangeInOtherAssets']
        },
        'Change in Accounts Payable': {
            'primary': 'ChangeInAccountsPayable',
            'alternatives': [],
            'children': []
        },
        'Change in Accrued Liabilities': {
            'primary': 'ChangeInAccruedLiabilities',
            'alternatives': ['ChangeInAccruedExpenses'],
            'children': ['ChangeInAccruedCompensation']
        },
        'Change in Deferred Revenue': {
            'primary': 'ChangeInDeferredRevenue',
            'alternatives': ['ChangeInContractLiabilities', 'ChangeInUnearnedRevenue'],
            'children': []
        },
        'Change in Income Taxes': {
            'primary': 'ChangeInIncomeTaxesPayableReceivable',
            'alternatives': ['ChangeInIncomeTaxPayable'],
            'children': []
        },
        'Other Operating Activities': {
            'primary': 'OtherOperatingActivitiesCashFlowAdjustments',
            'alternatives': [],
            'children': []
        },
        'Net Cash from Operating Activities': {
            'primary': 'NetCashProvidedByUsedInOperatingActivities',
            'alternatives': ['CashFlowsFromUsedInOperatingActivities'],
            'children': []
        }
    }
    
    # Investing Activities section
    investing_activities = {
        'Capital Expenditures': {
            'primary': 'PaymentsToAcquirePropertyPlantAndEquipment',
            'alternatives': ['CapitalExpenditures'],
            'children': []
        },
        'Proceeds from Asset Sales': {
            'primary': 'ProceedsFromSaleOfPropertyPlantAndEquipment',
            'alternatives': [],
            'children': []
        },
        'Purchases of Intangible Assets': {
            'primary': 'PaymentsToAcquireIntangibleAssetsExcludingGoodwill',
            'alternatives': [],
            'children': []
        },
        'Acquisitions, Net of Cash Acquired': {
            'primary': 'PaymentsToAcquireBusinessesNetOfCashAcquired',
            'alternatives': ['BusinessAcquisitionsNetCashAcquired'],
            'children': []
        },
        'Purchases of Investments': {
            'primary': 'PaymentsToAcquireInvestments',
            'alternatives': ['PurchaseOfInvestments'],
            'children': []
        },
        'Proceeds from Sale of Investments': {
            'primary': 'ProceedsFromSaleOfInvestments',
            'alternatives': [],
            'children': []
        },
        'Proceeds from Maturities of Investments': {
            'primary': 'ProceedsFromMaturitiesOfInvestments',
            'alternatives': ['ProceedsFromMaturityOfInvestments'],
            'children': []
        },
        'Other Investing Activities': {
            'primary': 'OtherInvestingActivities',
            'alternatives': [],
            'children': []
        },
        'Net Cash from Investing Activities': {
            'primary': 'NetCashProvidedByUsedInInvestingActivities',
            'alternatives': ['CashFlowsFromUsedInInvestingActivities'],
            'children': []
        }
    }
    
    # Financing Activities section
    financing_activities = {
        'Proceeds from Long-term Debt': {
            'primary': 'ProceedsFromIssuanceOfLongTermDebt',
            'alternatives': ['ProceedsFromDebtIssuance'],
            'children': []
        },
        'Repayment of Long-term Debt': {
            'primary': 'PaymentsOfPrincipalOnLongTermDebt',
            'alternatives': ['RepaymentsOfLongTermDebt', 'DebtRepayment'],
            'children': []
        },
        'Proceeds from Common Stock Issuance': {
            'primary': 'ProceedsFromIssuanceOfCommonStock',
            'alternatives': ['ProceedsFromStockPlans'],
            'children': []
        },
        'Common Stock Repurchases': {
            'primary': 'PaymentsForRepurchaseOfCommonStock',
            'alternatives': ['PaymentsToRepurchaseCommonStock', 'StockRepurchaseProgram'],
            'children': []
        },
        'Dividend Payments': {
            'primary': 'PaymentsOfDividends',
            'alternatives': ['DividendsPaid'],
            'children': []
        },
        'Lease Payments': {
            'primary': 'PaymentsOfFinanceLeaseLiabilities',
            'alternatives': ['PaymentsOfCapitalLeaseObligations'],
            'children': []
        },
        'Other Financing Activities': {
            'primary': 'OtherFinancingActivities',
            'alternatives': [],
            'children': []
        },
        'Net Cash from Financing Activities': {
            'primary': 'NetCashProvidedByUsedInFinancingActivities',
            'alternatives': ['CashFlowsFromUsedInFinancingActivities'],
            'children': []
        }
    }
    
    # Effect of Exchange Rate Changes and Net Change in Cash
    cash_reconciliation = {
        'Effect of Exchange Rate Changes': {
            'primary': 'EffectOfExchangeRateOnCashAndCashEquivalents',
            'alternatives': ['EffectOfExchangeRateChangesOnCash'],
            'children': []
        },
        'Net Change in Cash and Cash Equivalents': {
            'primary': 'IncreaseDecreaseInCashAndCashEquivalents',
            'alternatives': ['ChangeInCashAndCashEquivalents', 'CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalentsPeriodIncreaseDecrease'],
            'children': []
        },
        'Cash and Cash Equivalents, Beginning of Period': {
            'primary': 'CashAndCashEquivalentsAtCarryingValue',  # This will be filtered by context to beginning period
            'alternatives': ['CashAndCashEquivalentsPeriodStart', 'CashPeriodStart'],
            'children': []
        },
        'Cash and Cash Equivalents, End of Period': {
            'primary': 'CashAndCashEquivalentsAtCarryingValue',  # This will be filtered by context to ending period
            'alternatives': ['CashAndCashEquivalentsPeriodEnd', 'CashPeriodEnd'],
            'children': []
        }
    }
    
    # Supplemental information
    supplemental_info = {
        'Interest Paid': {
            'primary': 'InterestPaid',
            'alternatives': [],
            'children': []
        },
        'Income Taxes Paid': {
            'primary': 'IncomeTaxesPaid',
            'alternatives': [],
            'children': []
        }
    }
    
    # Helper function to find the best matching column
    def find_best_match(account_info, available_cols):
        # Check primary tag
        if account_info['primary'] in available_cols:
            return account_info['primary']
        
        # Check alternatives
        for alt in account_info['alternatives']:
            if alt in available_cols:
                return alt
                
        # Handle partial matches for more flexible matching
        primary_matches = []
        for col in available_cols:
            if account_info['primary'] in col:
                primary_matches.append(col)
        
        # If we found exactly one partial match for primary, return it
        if len(primary_matches) == 1:
            return primary_matches[0]
                
        # If we found multiple partial matches, try to find the best one
        if primary_matches:
            for match in primary_matches:
                # For cash flow statement, look for "CashFlow" or "NetCash" in the match
                if 'CashFlow' in match or 'NetCash' in match:
                    return match
            
            return primary_matches[0]  # Default to first one
        
        # Check for partial matches in alternatives
        for alt in account_info['alternatives']:
            for col in available_cols:
                if alt in col:
                    return col
        
        return None
    
    # Process each section
    def process_section(section_name, section_mapping):
        for account_name, account_info in section_mapping.items():
            # Special handling for Cash and Cash Equivalents at beginning/end of period
            if account_name == 'Cash and Cash Equivalents, Beginning of Period':
                # Look for beginning period cash
                for col in available_columns:
                    if 'CashAndCashEquivalents' in col and 'Begin' in col:
                        cash_flow[f"{section_name} - {account_name}"] = df[col]
                        break
                else:
                    # Try to find regular cash column with beginning period context
                    best_match = find_best_match(account_info, available_columns)
                    if best_match:
                        cash_flow[f"{section_name} - {account_name}"] = df[best_match]
                    else:
                        cash_flow[f"{section_name} - {account_name}"] = pd.Series(np.nan, index=df.index)
                continue
                
            if account_name == 'Cash and Cash Equivalents, End of Period':
                # Look for ending period cash
                for col in available_columns:
                    if 'CashAndCashEquivalents' in col and 'End' in col:
                        cash_flow[f"{section_name} - {account_name}"] = df[col]
                        break
                else:
                    # Try to find regular cash column with ending period context
                    best_match = find_best_match(account_info, available_columns)
                    if best_match:
                        cash_flow[f"{section_name} - {account_name}"] = df[best_match]
                    else:
                        cash_flow[f"{section_name} - {account_name}"] = pd.Series(np.nan, index=df.index)
                continue
            
            # Standard processing for other accounts
            best_match = find_best_match(account_info, available_columns)
            
            if best_match:
                # Direct match found
                cash_flow[f"{section_name} - {account_name}"] = df[best_match]
            elif account_info['children']:
                # Try to sum children if parent not found
                available_children = []
                for child in account_info['children']:
                    if child in available_columns:
                        available_children.append(child)
                    else:
                        # Look for variations
                        for col in available_columns:
                            if child in col:
                                available_children.append(col)
                
                if available_children:
                    cash_flow[f"{section_name} - {account_name}"] = df[available_children].sum(axis=1)
                else:
                    # No matches found
                    cash_flow[f"{section_name} - {account_name}"] = pd.Series(np.nan, index=df.index)
            else:
                # No matches found
                cash_flow[f"{section_name} - {account_name}"] = pd.Series(np.nan, index=df.index)
    
    # Process each section
    process_section("Operating", operating_activities)
    process_section("Investing", investing_activities)
    process_section("Financing", financing_activities)
    process_section("Cash", cash_reconciliation)
    process_section("Supplemental", supplemental_info)
    
    # Calculate missing values and add validation checks
    
    # If Net Cash from Operating Activities is missing, try to calculate it
    if ('Operating - Net Cash from Operating Activities' not in cash_flow.columns or 
        cash_flow['Operating - Net Cash from Operating Activities'].isna().all()):
        
        operating_cols = [col for col in cash_flow.columns 
                          if col.startswith('Operating - ') and 'Net Cash' not in col]
        
        if operating_cols and 'Operating - Net Income' in operating_cols:
            # Start with net income
            cash_flow['Operating - Net Cash from Operating Activities (calculated)'] = cash_flow['Operating - Net Income']
            
            # Add all other operating activities except net income
            for col in operating_cols:
                if col != 'Operating - Net Income':
                    # Skip NaN columns
                    if not cash_flow[col].isna().all():
                        cash_flow['Operating - Net Cash from Operating Activities (calculated)'] += cash_flow[col]
    
    # If Net Cash from Investing Activities is missing, try to calculate it
    if ('Investing - Net Cash from Investing Activities' not in cash_flow.columns or 
        cash_flow['Investing - Net Cash from Investing Activities'].isna().all()):
        
        investing_cols = [col for col in cash_flow.columns 
                          if col.startswith('Investing - ') and 'Net Cash' not in col]
        
        if investing_cols:
            cash_flow['Investing - Net Cash from Investing Activities (calculated)'] = 0
            
            # Sum all investing activities
            for col in investing_cols:
                # Skip NaN columns
                if not cash_flow[col].isna().all():
                    cash_flow['Investing - Net Cash from Investing Activities (calculated)'] += cash_flow[col]
    
    # If Net Cash from Financing Activities is missing, try to calculate it
    if ('Financing - Net Cash from Financing Activities' not in cash_flow.columns or 
        cash_flow['Financing - Net Cash from Financing Activities'].isna().all()):
        
        financing_cols = [col for col in cash_flow.columns 
                          if col.startswith('Financing - ') and 'Net Cash' not in col]
        
        if financing_cols:
            cash_flow['Financing - Net Cash from Financing Activities (calculated)'] = 0
            
            # Sum all financing activities
            for col in financing_cols:
                # Skip NaN columns
                if not cash_flow[col].isna().all():
                    cash_flow['Financing - Net Cash from Financing Activities (calculated)'] += cash_flow[col]
    
    # If Net Change in Cash is missing, calculate it from operating, investing, and financing
    if ('Cash - Net Change in Cash and Cash Equivalents' not in cash_flow.columns or 
        cash_flow['Cash - Net Change in Cash and Cash Equivalents'].isna().all()):
        
        # Choose the best available values for each section
        op_cash = ('Operating - Net Cash from Operating Activities' 
                   if 'Operating - Net Cash from Operating Activities' in cash_flow.columns 
                   else 'Operating - Net Cash from Operating Activities (calculated)')
        
        inv_cash = ('Investing - Net Cash from Investing Activities' 
                    if 'Investing - Net Cash from Investing Activities' in cash_flow.columns 
                    else 'Investing - Net Cash from Investing Activities (calculated)')
        
        fin_cash = ('Financing - Net Cash from Financing Activities' 
                    if 'Financing - Net Cash from Financing Activities' in cash_flow.columns 
                    else 'Financing - Net Cash from Financing Activities (calculated)')
        
        fx_effect = 'Cash - Effect of Exchange Rate Changes'
        
        # Calculate net change in cash if we have at least operating, investing, and financing
        cash_components = []
        for component in [op_cash, inv_cash, fin_cash]:
            if component in cash_flow.columns and not cash_flow[component].isna().all():
                cash_components.append(component)
                
        # Add forex effect if available
        if fx_effect in cash_flow.columns and not cash_flow[fx_effect].isna().all():
            cash_components.append(fx_effect)
            
        if len(cash_components) >= 3:  # At least have operating, investing, and financing
            cash_flow['Cash - Net Change in Cash and Cash Equivalents (calculated)'] = cash_flow[cash_components].sum(axis=1)
    
    # If Beginning Cash + Net Change doesn't equal Ending Cash, add validation check
    if ('Cash - Cash and Cash Equivalents, Beginning of Period' in cash_flow.columns and 
        'Cash - Cash and Cash Equivalents, End of Period' in cash_flow.columns):
        
        # Choose best available net change value
        net_change_col = ('Cash - Net Change in Cash and Cash Equivalents' 
                           if 'Cash - Net Change in Cash and Cash Equivalents' in cash_flow.columns 
                           else 'Cash - Net Change in Cash and Cash Equivalents (calculated)')
        
        if net_change_col in cash_flow.columns:
            cash_flow['Validation - Cash Reconciliation Difference'] = (
                cash_flow['Cash - Cash and Cash Equivalents, End of Period'] - 
                (cash_flow['Cash - Cash and Cash Equivalents, Beginning of Period'] + cash_flow[net_change_col])
            )
    
    # Add validation for operating + investing + financing = net change
    op_cash_col = next((col for col in cash_flow.columns 
                        if col.startswith('Operating - Net Cash') and not cash_flow[col].isna().all()), None)
    
    inv_cash_col = next((col for col in cash_flow.columns 
                         if col.startswith('Investing - Net Cash') and not cash_flow[col].isna().all()), None)
    
    fin_cash_col = next((col for col in cash_flow.columns 
                         if col.startswith('Financing - Net Cash') and not cash_flow[col].isna().all()), None)
    
    net_change_col = next((col for col in cash_flow.columns 
                           if col.startswith('Cash - Net Change') and not cash_flow[col].isna().all()), None)
    
    fx_effect_col = next((col for col in cash_flow.columns 
                          if 'Effect of Exchange Rate' in col and not cash_flow[col].isna().all()), None)
    
    if op_cash_col and inv_cash_col and fin_cash_col and net_change_col:
        components_sum = cash_flow[op_cash_col] + cash_flow[inv_cash_col] + cash_flow[fin_cash_col]
        
        # Add forex effect if available
        if fx_effect_col:
            components_sum += cash_flow[fx_effect_col]
            
        cash_flow['Validation - Net Cash Components Difference'] = cash_flow[net_change_col] - components_sum
    
    return cash_flow

# Function to display a human-readable cash flow statement
def display_cash_flow_statement(cash_flow, in_millions=True):
    """
    Display the cash flow statement in a clean, readable format.
    
    Args:
        cash_flow: DataFrame with the reconstructed cash flow statement
        in_millions: If True, display values in millions; otherwise in thousands
    
    Returns:
        DataFrame with the formatted cash flow statement
    """
    # Make a copy to avoid modifying the original
    formatted_cf = cash_flow.copy()
    
    # Identify numeric columns
    numeric_cols = [col for col in formatted_cf.columns 
                    if any(col.startswith(prefix) for prefix in ['Operating', 'Investing', 'Financing', 'Cash', 'Supplemental', 'Validation'])]
    
    # Convert to millions or thousands
    divisor = 1_000_000 if in_millions else 1_000
    suffix = 'M' if in_millions else 'K'
    
    for col in numeric_cols:
        formatted_cf[col] = formatted_cf[col] / divisor
        
    # Format the date column if it exists
    if 'end' in formatted_cf.columns:
        try:
            formatted_cf['end'] = pd.to_datetime(formatted_cf['end']).dt.strftime('%Y-%m-%d')
        except:
            pass
    
    # Create sections for display in proper cash flow statement order
    display_sections = [
        'Operating',
        'Investing',
        'Financing',
        'Cash',
        'Supplemental'
    ]
    
    # Columns to display
    display_cols = ['end'] if 'end' in formatted_cf.columns else []
    for section in display_sections:
        section_cols = [col for col in formatted_cf.columns if col.startswith(f"{section} - ")]
        display_cols.extend(section_cols)
    
    # Add validation columns at the end if they exist
    validation_cols = [col for col in formatted_cf.columns if col.startswith('Validation - ')]
    display_cols.extend(validation_cols)
    
    return formatted_cf[display_cols]

# Function to remove columns that only contain NaN values
def remove_nan_only_columns(df):
    """
    Removes columns that contain only NaN values.
    
    Args:
        df: DataFrame to clean
    
    Returns:
        DataFrame with NaN-only columns removed
    """
    # Find columns where all values are NaN
    nan_cols = df.columns[df.isna().all()].tolist()
    
    # Drop those columns
    return df.drop(columns=nan_cols)

# Example usage
def analyze_microsoft_cash_flow(df):
    """
    Analyze Microsoft's cash flow statement from XBRL data.
    
    Args:
        df: DataFrame containing Microsoft's XBRL-tagged financial data
    
    Returns:
        The reconstructed cash flow statement
    """
    print("Reconstructing Microsoft's cash flow statement from XBRL data...")
    cash_flow = reconstruct_cash_flow_statement(df)
    
    # Remove columns with only NaN values
    cash_flow = remove_nan_only_columns(cash_flow)
    
    print("\nCash Flow Statement Summary:")
    print(f"- Total columns: {len(cash_flow.columns)}")
    print(f"- Data for {len(cash_flow)} reporting periods")
    
    # Display the cash flow statement
    print("\nFormatted Cash Flow Statement (in millions USD):")
    formatted_cf = display_cash_flow_statement(cash_flow, in_millions=True)
    
    return cash_flow, formatted_cf

# To use this code with your DataFrame:
cash_flow, formatted_cf = analyze_microsoft_cash_flow(consolidated_df)

Reconstructing Microsoft's cash flow statement from XBRL data...

Cash Flow Statement Summary:
- Total columns: 25
- Data for 26 reporting periods

Formatted Cash Flow Statement (in millions USD):


In [54]:
# Keep rows with exactly Q1, Q2, Q3, Q4 or just the year (like CY2019)
cash_flow = cash_flow[~cash_flow['frame'].str.match(r'CY\d{4}Q[1-4].+')]
cash_flow = cash_flow.sort_values(by='end')
cash_flow.T

,48,52,53,55,56,57,58,60,61,62,...,72,73,74,76,77,78,79,81,82,83
filed,2019-04-26,2019-02-01,2019-04-26,2019-07-26,2019-10-25,2020-01-31,2021-04-30,2021-07-30,2020-10-30,2021-02-03,...,2022-10-28,2023-02-03,2024-05-01,2023-08-04,2023-10-27,2024-02-02,2024-05-01,2024-08-02,2024-11-01,2025-02-07
company_name,"AMAZON.COM, INC.","AMAZON.COM, INC.","AMAZON.COM, INC.","AMAZON.COM, INC.","AMAZON.COM, INC.","AMAZON.COM, INC.","AMAZON.COM, INC.","AMAZON.COM, INC.","AMAZON.COM, INC.","AMAZON.COM, INC.",...,"AMAZON.COM, INC.","AMAZON.COM, INC.","AMAZON.COM, INC.","AMAZON.COM, INC.","AMAZON.COM, INC.","AMAZON.COM, INC.","AMAZON.COM, INC.","AMAZON.COM, INC.","AMAZON.COM, INC.","AMAZON.COM, INC."
end,2018-03-31,2018-12-31,2019-03-31,2019-06-30,2019-09-30,2019-12-31,2020-03-31,2020-06-30,2020-09-30,2020-12-31,...,2022-09-30,2022-12-31,2023-03-31,2023-06-30,2023-09-30,2023-12-31,2024-03-31,2024-06-30,2024-09-30,2024-12-31
unit,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD,...,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD
form,10-Q,10-K,10-Q,10-Q,10-Q,10-K,10-Q,10-Q,10-Q,10-K,...,10-Q,10-K,10-Q,10-Q,10-Q,10-K,10-Q,10-Q,10-Q,10-K
frame,CY2017,CY2018,CY2019Q1,CY2019Q2,CY2019Q3,CY2019,CY2019,CY2020Q2,CY2020Q3,CY2020,...,CY2022,CY2022,CY2022,CY2023Q2,CY2023Q3,CY2023,CY2023,CY2024Q2,CY2024,CY2024
cik,1018724,1018724,1018724,1018724,1018724,1018724,1018724,1018724,1018724,1018724,...,1018724,1018724,1018724,1018724,1018724,1018724,1018724,1018724,1018724,1018724
Operating - Net Income,1629000000.0,10073000000.0,3561000000.0,2625000000.0,2134000000.0,11588000000.0,2535000000.0,5243000000.0,6331000000.0,21331000000.0,...,2872000000.0,-2722000000.0,3172000000.0,6750000000.0,9879000000.0,30425000000.0,10431000000.0,13485000000.0,15328000000.0,59248000000.0
Operating - Depreciation and Amortization,NaN,NaN,NaN,NaN,NaN,46975000000.0,NaN,NaN,NaN,60434000000.0,...,NaN,97015000000.0,NaN,NaN,NaN,120111000000.0,NaN,NaN,NaN,141390000000.0
Operating - Stock-based Compensation,1182000000.0,5418000000.0,1274000000.0,1971000000.0,1779000000.0,6864000000.0,1757000000.0,2601000000.0,2288000000.0,9208000000.0,...,5556000000.0,19621000000.0,4748000000.0,7127000000.0,5829000000.0,24023000000.0,4961000000.0,6722000000.0,5333000000.0,22011000000.0


In [55]:
balance_sheet.head(10)

,filed,company_name,end,unit,form,frame,cik,Assets - Cash and Cash Equivalents,Assets - Short-term Investments,"Assets - Accounts Receivable, Net (Current)",...,Liabilities - Total Liabilities,Equity - Common Stock and Paid-in Capital,Equity - Retained Earnings,Equity - Accumulated Other Comprehensive Income,Equity - Total Stockholders Equity,Equity - Total Liabilities and Equity,Validation - Current Assets Difference,Validation - Current Liabilities Difference,Validation - A = L+E Difference,Validation - Components = Total L+E Difference
48,2019-04-26,"AMAZON.COM, INC.",2018-03-31,USD,10-Q,CY2017,1018724,1.667600e+10,8.287000e+09,1.202600e+10,...,9.489900e+10,2.256800e+10,1.119900e+10,-4.670000e+08,3.146300e+10,1.263620e+11,3.880300e+10,4.799200e+10,0.0,0.000000e+00
52,2019-02-01,"AMAZON.COM, INC.",2018-12-31,USD,10-K,CY2018,1018724,3.175000e+10,9.500000e+09,1.667700e+10,...,7.490000e+08,2.679600e+10,1.962500e+10,-1.035000e+09,4.354900e+10,1.626480e+11,5.842400e+10,6.702000e+10,0.0,-1.183500e+11
53,2019-04-26,"AMAZON.COM, INC.",2019-03-31,USD,10-Q,CY2019Q1,1018724,2.311500e+10,1.390500e+10,1.597900e+10,...,1.296920e+11,2.806400e+10,2.319300e+10,-1.010000e+09,4.841000e+10,1.781020e+11,5.345200e+10,6.208700e+10,0.0,0.000000e+00
55,2019-07-26,"AMAZON.COM, INC.",2019-06-30,USD,10-Q,CY2019Q2,1018724,2.261600e+10,1.884700e+10,1.674700e+10,...,1.382900e+11,3.004000e+10,2.581800e+10,-9.600000e+08,5.306100e+10,1.913510e+11,6.004300e+10,6.804200e+10,0.0,0.000000e+00
56,2019-10-25,"AMAZON.COM, INC.",2019-09-30,USD,10-Q,CY2019Q3,1018724,2.325500e+10,2.014600e+10,1.688700e+10,...,1.425910e+11,3.182200e+10,2.795200e+10,-1.429000e+09,5.650800e+10,1.990990e+11,6.216700e+10,6.929500e+10,0.0,0.000000e+00
57,2020-01-31,"AMAZON.COM, INC.",2019-12-31,USD,10-K,CY2019,1018724,3.609200e+10,1.892900e+10,2.081600e+10,...,6.400000e+08,3.366300e+10,3.122000e+10,-9.860000e+08,6.206000e+10,2.252480e+11,7.551800e+10,8.650700e+10,0.0,-1.625480e+11
58,2021-04-30,"AMAZON.COM, INC.",2020-03-31,USD,10-Q,CY2019,1018724,2.720100e+10,2.209100e+10,1.783600e+10,...,1.559660e+11,3.541700e+10,3.375500e+10,-2.063000e+09,6.527200e+10,2.212380e+11,6.814900e+10,7.840000e+10,0.0,0.000000e+00
60,2021-07-30,"AMAZON.COM, INC.",2020-06-30,USD,10-Q,CY2020Q2,1018724,3.746600e+10,3.392500e+10,1.991800e+10,...,1.845860e+11,3.802200e+10,3.899800e+10,-1.455000e+09,7.372800e+10,2.583140e+11,9.099000e+10,9.274200e+10,0.0,0.000000e+00
61,2020-10-30,"AMAZON.COM, INC.",2020-09-30,USD,10-Q,CY2020Q3,1018724,2.993000e+10,3.847200e+10,2.083200e+10,...,1.994040e+11,4.031200e+10,4.532900e+10,-1.029000e+09,8.277500e+10,2.821790e+11,9.213700e+10,1.017570e+11,0.0,0.000000e+00
62,2021-02-03,"AMAZON.COM, INC.",2020-12-31,USD,10-K,CY2020,1018724,4.212200e+10,4.227400e+10,2.454200e+10,...,8.930000e+08,4.287000e+10,5.255100e+10,-1.800000e+08,9.340400e+10,3.211950e+11,1.081910e+11,1.252300e+11,0.0,-2.268980e+11


In [56]:
balance_sheet.to_csv('./MSFT_bs.csv')
income_statement.to_csv('./MSFT_is.csv')

In [57]:
income_statement.head(10)

,filed,company_name,end,unit,form,frame,cik,Revenue - Total Revenue,Cost - Total Cost of Revenue,Operating Expenses - Sales and Marketing,...,Other - Interest Income,Other - Interest Expense,Other - Foreign Currency Transaction Gain/Loss,"Other - Other Income/Expense, Net",Tax - Income Tax Expense/Benefit,Net Income - Net Income,EPS - Basic EPS,EPS - Diluted EPS,Gross - Gross Profit (calculated),Income Before Tax - Income Before Income Taxes (calculated)
48,2019-04-26,"AMAZON.COM, INC.",2018-03-31,USD,10-Q,CY2017,1018724,5.104200e+10,3.073500e+10,2.699000e+09,...,80000000.0,3.300000e+08,NaN,-1.100000e+07,2.870000e+08,1.629000e+09,3.36,3.27,2.030700e+10,1.916000e+09
52,2019-02-01,"AMAZON.COM, INC.",2018-12-31,USD,10-K,CY2018,1018724,2.328870e+11,1.391560e+11,1.381400e+10,...,440000000.0,1.417000e+09,-206000000.0,-1.160000e+09,1.197000e+09,1.007300e+10,20.68,20.14,9.373100e+10,1.126100e+10
53,2019-04-26,"AMAZON.COM, INC.",2019-03-31,USD,10-Q,CY2019Q1,1018724,5.970000e+10,3.392000e+10,3.664000e+09,...,183000000.0,3.660000e+08,NaN,-1.900000e+07,8.360000e+08,3.561000e+09,7.24,7.09,2.578000e+10,4.401000e+09
55,2019-07-26,"AMAZON.COM, INC.",2019-06-30,USD,10-Q,CY2019Q2,1018724,6.340400e+10,3.633700e+10,4.291000e+09,...,215000000.0,3.830000e+08,NaN,-1.950000e+08,2.570000e+08,2.625000e+09,5.32,5.22,2.706700e+10,2.889000e+09
56,2019-10-25,"AMAZON.COM, INC.",2019-09-30,USD,10-Q,CY2019Q3,1018724,6.998100e+10,4.130200e+10,4.752000e+09,...,224000000.0,3.960000e+08,NaN,-5.250000e+08,4.940000e+08,2.134000e+09,4.31,4.23,2.867900e+10,2.632000e+09
57,2020-01-31,"AMAZON.COM, INC.",2019-12-31,USD,10-K,CY2019,1018724,2.805220e+11,1.655360e+11,1.887800e+10,...,832000000.0,1.600000e+09,-20000000.0,-5.650000e+08,2.374000e+09,1.158800e+10,23.46,23.01,1.149860e+11,1.397600e+10
58,2021-04-30,"AMAZON.COM, INC.",2020-03-31,USD,10-Q,CY2019,1018724,7.545200e+10,4.425700e+10,4.828000e+09,...,202000000.0,4.020000e+08,-222000000.0,-6.060000e+08,7.440000e+08,2.535000e+09,5.09,5.01,3.119500e+10,3.383000e+09
60,2021-07-30,"AMAZON.COM, INC.",2020-06-30,USD,10-Q,CY2020Q2,1018724,8.891200e+10,5.266000e+10,4.345000e+09,...,135000000.0,4.030000e+08,13000000.0,3.780000e+08,9.840000e+08,5.243000e+09,10.50,10.30,3.625200e+10,6.221000e+09
61,2020-10-30,"AMAZON.COM, INC.",2020-09-30,USD,10-Q,CY2020Q3,1018724,9.614500e+10,5.710600e+10,5.434000e+09,...,118000000.0,4.280000e+08,112000000.0,6.150000e+08,5.690000e+08,6.331000e+09,12.63,12.37,3.903900e+10,6.809000e+09
62,2021-02-03,"AMAZON.COM, INC.",2020-12-31,USD,10-K,CY2020,1018724,3.860640e+11,2.333070e+11,2.200800e+10,...,555000000.0,1.647000e+09,35000000.0,1.279000e+09,2.863000e+09,2.133100e+10,2.13,2.09,1.527570e+11,2.417800e+10


In [58]:
cash_flow.T

,48,52,53,55,56,57,58,60,61,62,...,72,73,74,76,77,78,79,81,82,83
filed,2019-04-26,2019-02-01,2019-04-26,2019-07-26,2019-10-25,2020-01-31,2021-04-30,2021-07-30,2020-10-30,2021-02-03,...,2022-10-28,2023-02-03,2024-05-01,2023-08-04,2023-10-27,2024-02-02,2024-05-01,2024-08-02,2024-11-01,2025-02-07
company_name,"AMAZON.COM, INC.","AMAZON.COM, INC.","AMAZON.COM, INC.","AMAZON.COM, INC.","AMAZON.COM, INC.","AMAZON.COM, INC.","AMAZON.COM, INC.","AMAZON.COM, INC.","AMAZON.COM, INC.","AMAZON.COM, INC.",...,"AMAZON.COM, INC.","AMAZON.COM, INC.","AMAZON.COM, INC.","AMAZON.COM, INC.","AMAZON.COM, INC.","AMAZON.COM, INC.","AMAZON.COM, INC.","AMAZON.COM, INC.","AMAZON.COM, INC.","AMAZON.COM, INC."
end,2018-03-31,2018-12-31,2019-03-31,2019-06-30,2019-09-30,2019-12-31,2020-03-31,2020-06-30,2020-09-30,2020-12-31,...,2022-09-30,2022-12-31,2023-03-31,2023-06-30,2023-09-30,2023-12-31,2024-03-31,2024-06-30,2024-09-30,2024-12-31
unit,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD,...,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD
form,10-Q,10-K,10-Q,10-Q,10-Q,10-K,10-Q,10-Q,10-Q,10-K,...,10-Q,10-K,10-Q,10-Q,10-Q,10-K,10-Q,10-Q,10-Q,10-K
frame,CY2017,CY2018,CY2019Q1,CY2019Q2,CY2019Q3,CY2019,CY2019,CY2020Q2,CY2020Q3,CY2020,...,CY2022,CY2022,CY2022,CY2023Q2,CY2023Q3,CY2023,CY2023,CY2024Q2,CY2024,CY2024
cik,1018724,1018724,1018724,1018724,1018724,1018724,1018724,1018724,1018724,1018724,...,1018724,1018724,1018724,1018724,1018724,1018724,1018724,1018724,1018724,1018724
Operating - Net Income,1629000000.0,10073000000.0,3561000000.0,2625000000.0,2134000000.0,11588000000.0,2535000000.0,5243000000.0,6331000000.0,21331000000.0,...,2872000000.0,-2722000000.0,3172000000.0,6750000000.0,9879000000.0,30425000000.0,10431000000.0,13485000000.0,15328000000.0,59248000000.0
Operating - Depreciation and Amortization,NaN,NaN,NaN,NaN,NaN,46975000000.0,NaN,NaN,NaN,60434000000.0,...,NaN,97015000000.0,NaN,NaN,NaN,120111000000.0,NaN,NaN,NaN,141390000000.0
Operating - Stock-based Compensation,1182000000.0,5418000000.0,1274000000.0,1971000000.0,1779000000.0,6864000000.0,1757000000.0,2601000000.0,2288000000.0,9208000000.0,...,5556000000.0,19621000000.0,4748000000.0,7127000000.0,5829000000.0,24023000000.0,4961000000.0,6722000000.0,5333000000.0,22011000000.0


## I want to merge all of the accounts. This way when I am computing the different ratios all of the different accounts and cross balance sheet income statement accoutns are in the same dataframe.

In [59]:
# Pre-Processing: Clean columns names
def clean_col_names(df):
    """Removes leading/trailing whitespace from DataFrame column names."""
    df.columns = df.columns.str.strip()
    print(f"Cleaned columns: {df.columns.tolist()}") # Optional: verify cleaning
    return df

balance_sheet = clean_col_names(balance_sheet)
income_statement = clean_col_names(income_statement)
cash_flow = clean_col_names(cash_flow)


Cleaned columns: ['filed', 'company_name', 'end', 'unit', 'form', 'frame', 'cik', 'Assets - Cash and Cash Equivalents', 'Assets - Short-term Investments', 'Assets - Accounts Receivable, Net (Current)', 'Assets - Inventory', 'Assets - Other Current Assets', 'Assets - Total Current Assets', 'Assets - Property, Plant and Equipment, Net', 'Assets - Goodwill', 'Assets - Intangible Assets, Net', 'Assets - Other Long-term Assets', 'Assets - Total Assets', 'Liabilities - Accounts Payable', 'Liabilities - Current Portion of Long-term Debt', 'Liabilities - Total Current Liabilities', 'Liabilities - Long-term Debt', 'Liabilities - Operating Lease Liabilities (Non-current)', 'Liabilities - Other Long-term Liabilities', 'Liabilities - Total Liabilities', 'Equity - Common Stock and Paid-in Capital', 'Equity - Retained Earnings', 'Equity - Accumulated Other Comprehensive Income', 'Equity - Total Stockholders Equity', 'Equity - Total Liabilities and Equity', 'Validation - Current Assets Difference', '

In [60]:
# Define the merge key 
# The merge key is the frame column because this column has the account data for Calendar Year or Calendar Period for different quarters
merge_key = 'frame'

# Step 1: Merge balance_sheet and income_statement
# Use 'outer' merge to keep all unique 'frame' values from both DataFrames.
# Add suffixes to distinguish overlapping columns (other than the merge key).
merged_df_1 = pd.merge(balance_sheet, income_statement,
                       on=merge_key,
                       how='outer',
                       suffixes=('_bs', '_is'))

# Doing the final merge to have the cashflow statement together with the balance sheet and income statement
merged_df_final = pd.merge(merged_df_1, cash_flow,
                           on=merge_key,
                           how='outer',
                           suffixes=('_merged', '_cf')) # Suffixes apply to cols overlapping between merged_df_1 & cash_flow






In [61]:
merged_df_final = clean_col_names(merged_df_final)

Cleaned columns: ['filed_bs', 'company_name_bs', 'end_bs', 'unit_bs', 'form_bs', 'frame', 'cik_bs', 'Assets - Cash and Cash Equivalents', 'Assets - Short-term Investments', 'Assets - Accounts Receivable, Net (Current)', 'Assets - Inventory', 'Assets - Other Current Assets', 'Assets - Total Current Assets', 'Assets - Property, Plant and Equipment, Net', 'Assets - Goodwill', 'Assets - Intangible Assets, Net', 'Assets - Other Long-term Assets', 'Assets - Total Assets', 'Liabilities - Accounts Payable', 'Liabilities - Current Portion of Long-term Debt', 'Liabilities - Total Current Liabilities', 'Liabilities - Long-term Debt', 'Liabilities - Operating Lease Liabilities (Non-current)', 'Liabilities - Other Long-term Liabilities', 'Liabilities - Total Liabilities', 'Equity - Common Stock and Paid-in Capital', 'Equity - Retained Earnings', 'Equity - Accumulated Other Comprehensive Income', 'Equity - Total Stockholders Equity', 'Equity - Total Liabilities and Equity', 'Validation - Current Ass

In [62]:
import pandas as pd
import numpy as np

def calculate_cross_statement_ratios(df):
    """
    Calculate financial ratios that use both balance sheet and income statement accounts.
    For a single company with data already sorted chronologically.
    
    Parameters:
    df (DataFrame): DataFrame containing both balance sheet and income statement accounts
    
    Returns:
    DataFrame: Original DataFrame with added ratio columns
    """
    # Create a copy of the DataFrame to avoid modifying the original
    result_df = df.copy()
    
    # Define a function to safely calculate ratios, handling division by zero and NaN
    def safe_divide(numerator, denominator):
        # Perform the division
        result = numerator / denominator
        # Replace infinite values (from division by zero) with NaN
        result.replace([np.inf, -np.inf], np.nan, inplace=True)
        return result
    
    # Define a function to check if all required columns exist
    def columns_exist(column_list):
        return all(column in df.columns for column in column_list)
    
    # Track which calculations were performed and which were skipped
    calculated_items = []
    skipped_items = []
    
    # Define previous period values to calculate
    previous_period_values = [
        {
            'name': 'Prev_Assets_Total_Assets',
            'source': 'Assets - Total Assets',
            'description': 'Previous Period Total Assets'
        },
        {
            'name': 'Prev_Equity_Total_Stockholders_Equity',
            'source': 'Equity - Total Stockholders Equity',
            'description': 'Previous Period Total Stockholders Equity'
        },
        {
            'name': 'Prev_Assets_Inventory',
            'source': 'Assets - Inventory',
            'description': 'Previous Period Inventory'
        },
        {
            'name': 'Prev_Assets_Accounts_Receivable_Current',
            'source': 'Assets - Accounts Receivable, Net (Current)',
            'description': 'Previous Period Accounts Receivable'
        },
        {
            'name': 'Prev_Liabilities_Accounts_Payable',
            'source': 'Liabilities - Accounts Payable',
            'description': 'Previous Period Accounts Payable'
        },
        {
            'name': 'Prev_Assets_Total_Current_Assets',
            'source': 'Assets - Total Current Assets',
            'description': 'Previous Period Total Current Assets'
        },
        {
            'name': 'Prev_Liabilities_Total_Current_Liabilities',
            'source': 'Liabilities - Total Current Liabilities',
            'description': 'Previous Period Total Current Liabilities'
        }
    ]
    
    # Calculate previous period values if columns exist
    for item in previous_period_values:
        if columns_exist([item['source']]):
            result_df[item['name']] = result_df[item['source']].shift(1)
            calculated_items.append(item['name'])
        else:
            skipped_items.append(f"{item['name']} ({item['description']})")
    
    # Define average values to calculate
    average_values = [
        {
            'name': 'Avg_Total_Assets',
            'current': 'Assets - Total Assets',
            'previous': 'Prev_Assets_Total_Assets',
            'description': 'Average Total Assets'
        },
        {
            'name': 'Avg_Stockholders_Equity',
            'current': 'Equity - Total Stockholders Equity',
            'previous': 'Prev_Equity_Total_Stockholders_Equity',
            'description': 'Average Stockholders Equity'
        },
        {
            'name': 'Avg_Inventory',
            'current': 'Assets - Inventory',
            'previous': 'Prev_Assets_Inventory',
            'description': 'Average Inventory'
        },
        {
            'name': 'Avg_Accounts_Receivable',
            'current': 'Assets - Accounts Receivable, Net (Current)',
            'previous': 'Prev_Assets_Accounts_Receivable_Current',
            'description': 'Average Accounts Receivable'
        },
        {
            'name': 'Avg_Accounts_Payable',
            'current': 'Liabilities - Accounts Payable',
            'previous': 'Prev_Liabilities_Accounts_Payable',
            'description': 'Average Accounts Payable'
        }
    ]
    
    # Calculate average values if columns exist
    for item in average_values:
        if columns_exist([item['current']]) and item['previous'] in result_df.columns:
            result_df[item['name']] = (result_df[item['current']] + result_df[item['previous']]) / 2
            calculated_items.append(item['name'])
        else:
            skipped_items.append(f"{item['name']} ({item['description']})")
    
    # Working Capital calculations
    if columns_exist(['Assets - Total Current Assets', 'Liabilities - Total Current Liabilities']):
        result_df['Working_Capital'] = result_df['Assets - Total Current Assets'] - result_df['Liabilities - Total Current Liabilities']
        calculated_items.append('Working_Capital')
        
        if all(col in result_df.columns for col in ['Prev_Assets_Total_Current_Assets', 'Prev_Liabilities_Total_Current_Liabilities']):
            result_df['Prev_Working_Capital'] = result_df['Prev_Assets_Total_Current_Assets'] - result_df['Prev_Liabilities_Total_Current_Liabilities']
            result_df['Avg_Working_Capital'] = (result_df['Working_Capital'] + result_df['Prev_Working_Capital']) / 2
            calculated_items.extend(['Prev_Working_Capital', 'Avg_Working_Capital'])
        else:
            skipped_items.append('Prev_Working_Capital (Previous Period Working Capital)')
            skipped_items.append('Avg_Working_Capital (Average Working Capital)')
    else:
        skipped_items.append('Working_Capital (Current Assets - Current Liabilities)')
        skipped_items.append('Prev_Working_Capital (Previous Period Working Capital)')
        skipped_items.append('Avg_Working_Capital (Average Working Capital)')
    
    # Total Debt calculation
    if columns_exist(['Liabilities - Current Portion of Long-term Debt', 'Liabilities - Long-term Debt']):
        result_df['Total_Debt'] = result_df['Liabilities - Current Portion of Long-term Debt'] + result_df['Liabilities - Long-term Debt']
        calculated_items.append('Total_Debt')
    else:
        skipped_items.append('Total_Debt (Current Portion of Long-term Debt + Long-term Debt)')
    
    # Define ratios to calculate
    ratios = [
        {
            'name': 'Return_on_Assets',
            'numerator': 'Net Income - Net Income',
            'denominator': 'Avg_Total_Assets',
            'description': 'Net Income / Average Total Assets'
        },
        {
            'name': 'Return_on_Equity',
            'numerator': 'Net Income - Net Income',
            'denominator': 'Avg_Stockholders_Equity',
            'description': 'Net Income / Average Stockholders Equity'
        },
        {
            'name': 'Asset_Turnover_Ratio',
            'numerator': 'Revenue - Total Revenue',
            'denominator': 'Avg_Total_Assets',
            'description': 'Revenue / Average Total Assets'
        },
        {
            'name': 'Inventory_Turnover_Ratio',
            'numerator': 'Cost - Total Cost of Revenue',
            'denominator': 'Avg_Inventory',
            'description': 'Cost of Revenue / Average Inventory'
        },
        {
            'name': 'Accounts_Receivable_Turnover_Ratio',
            'numerator': 'Revenue - Total Revenue',
            'denominator': 'Avg_Accounts_Receivable',
            'description': 'Revenue / Average Accounts Receivable'
        },
        {
            'name': 'Accounts_Payable_Turnover_Ratio',
            'numerator': 'Cost - Total Cost of Revenue',
            'denominator': 'Avg_Accounts_Payable',
            'description': 'Cost of Revenue / Average Accounts Payable'
        },
        {
            'name': 'Working_Capital_Turnover_Ratio',
            'numerator': 'Revenue - Total Revenue',
            'denominator': 'Avg_Working_Capital',
            'description': 'Revenue / Average Working Capital'
        },
        {
            'name': 'Debt_to_EBITDA_Ratio',
            'numerator': 'Total_Debt',
            'denominator': 'Operating - Operating Income',
            'description': 'Total Debt / Operating Income'
        }
    ]
    
    # Calculate each ratio if the required columns exist
    for ratio in ratios:
        # Check if numerator exists in original DataFrame or was calculated
        numerator_exists = ratio['numerator'] in df.columns or ratio['numerator'] in result_df.columns
        # Check if denominator exists in original DataFrame or was calculated
        denominator_exists = ratio['denominator'] in df.columns or ratio['denominator'] in result_df.columns
        
        if numerator_exists and denominator_exists:
            result_df[ratio['name']] = safe_divide(
                result_df[ratio['numerator']],
                result_df[ratio['denominator']]
            )
            calculated_items.append(ratio['name'])
        else:
            skipped_items.append(f"{ratio['name']} ({ratio['description']})")
    
    # Print summary
    if calculated_items:
        print(f"Successfully calculated: {', '.join(calculated_items)}")
    if skipped_items:
        print(f"Skipped due to missing columns: {', '.join(skipped_items)}")
    
    return result_df

# Example usage:
# Assuming 'financial_data' is a DataFrame containing both balance sheet and income statement accounts
financial_data_with_ratios = calculate_cross_statement_ratios(merged_df_final)
financial_data_with_ratios.head()

Successfully calculated: Prev_Assets_Total_Assets, Prev_Equity_Total_Stockholders_Equity, Prev_Assets_Inventory, Prev_Assets_Accounts_Receivable_Current, Prev_Liabilities_Accounts_Payable, Prev_Assets_Total_Current_Assets, Prev_Liabilities_Total_Current_Liabilities, Avg_Total_Assets, Avg_Stockholders_Equity, Avg_Inventory, Avg_Accounts_Receivable, Avg_Accounts_Payable, Working_Capital, Prev_Working_Capital, Avg_Working_Capital, Total_Debt, Return_on_Assets, Return_on_Equity, Asset_Turnover_Ratio, Inventory_Turnover_Ratio, Accounts_Receivable_Turnover_Ratio, Accounts_Payable_Turnover_Ratio, Working_Capital_Turnover_Ratio, Debt_to_EBITDA_Ratio


,filed_bs,company_name_bs,end_bs,unit_bs,form_bs,frame,cik_bs,Assets - Cash and Cash Equivalents,Assets - Short-term Investments,"Assets - Accounts Receivable, Net (Current)",...,Avg_Working_Capital,Total_Debt,Return_on_Assets,Return_on_Equity,Asset_Turnover_Ratio,Inventory_Turnover_Ratio,Accounts_Receivable_Turnover_Ratio,Accounts_Payable_Turnover_Ratio,Working_Capital_Turnover_Ratio,Debt_to_EBITDA_Ratio
0,2019-04-26,"AMAZON.COM, INC.",2018-03-31,USD,10-Q,CY2017,1018724,1.667600e+10,8.287000e+09,1.202600e+10,...,NaN,2.469300e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.814219
1,2019-02-01,"AMAZON.COM, INC.",2018-12-31,USD,10-K,CY2018,1018724,3.175000e+10,9.500000e+09,1.667700e+10,...,4.747000e+09,2.486600e+10,0.069707,0.268570,1.611619,8.973754,16.227363,4.392273,49.059827,2.001932
2,2020-01-31,"AMAZON.COM, INC.",2019-12-31,USD,10-K,CY2019,1018724,3.609200e+10,1.892900e+10,2.081600e+10,...,7.616000e+09,2.471900e+10,0.059748,0.219451,1.446377,8.788511,14.963967,3.877857,36.833246,1.699952
3,2020-01-31,"AMAZON.COM, INC.",2019-12-31,USD,10-K,CY2019,1018724,3.609200e+10,1.892900e+10,2.081600e+10,...,8.522000e+09,2.471900e+10,0.051446,0.186723,1.245392,8.076109,13.476268,3.508382,32.917390,1.699952
4,2020-01-31,"AMAZON.COM, INC.",2019-12-31,USD,10-K,CY2019,1018724,3.609200e+10,1.892900e+10,2.081600e+10,...,8.522000e+09,2.471900e+10,0.011254,0.040848,0.334973,2.159194,3.624712,0.937986,8.853790,6.196791


In [63]:
# Example usage:
financial_data_with_ratios = calculate_cross_statement_ratios(merged_df_final)


Successfully calculated: Prev_Assets_Total_Assets, Prev_Equity_Total_Stockholders_Equity, Prev_Assets_Inventory, Prev_Assets_Accounts_Receivable_Current, Prev_Liabilities_Accounts_Payable, Prev_Assets_Total_Current_Assets, Prev_Liabilities_Total_Current_Liabilities, Avg_Total_Assets, Avg_Stockholders_Equity, Avg_Inventory, Avg_Accounts_Receivable, Avg_Accounts_Payable, Working_Capital, Prev_Working_Capital, Avg_Working_Capital, Total_Debt, Return_on_Assets, Return_on_Equity, Asset_Turnover_Ratio, Inventory_Turnover_Ratio, Accounts_Receivable_Turnover_Ratio, Accounts_Payable_Turnover_Ratio, Working_Capital_Turnover_Ratio, Debt_to_EBITDA_Ratio


In [64]:
financial_data_with_ratios.to_csv('./ratios.csv')

In [65]:
merged_df_final.columns

Index(['filed_bs', 'company_name_bs', 'end_bs', 'unit_bs', 'form_bs', 'frame',
       'cik_bs', 'Assets - Cash and Cash Equivalents',
       'Assets - Short-term Investments',
       'Assets - Accounts Receivable, Net (Current)', 'Assets - Inventory',
       'Assets - Other Current Assets', 'Assets - Total Current Assets',
       'Assets - Property, Plant and Equipment, Net', 'Assets - Goodwill',
       'Assets - Intangible Assets, Net', 'Assets - Other Long-term Assets',
       'Assets - Total Assets', 'Liabilities - Accounts Payable',
       'Liabilities - Current Portion of Long-term Debt',
       'Liabilities - Total Current Liabilities',
       'Liabilities - Long-term Debt',
       'Liabilities - Operating Lease Liabilities (Non-current)',
       'Liabilities - Other Long-term Liabilities',
       'Liabilities - Total Liabilities',
       'Equity - Common Stock and Paid-in Capital',
       'Equity - Retained Earnings',
       'Equity - Accumulated Other Comprehensive Income',
   

### Cleaning up the duplicate metadata columns created during the merge process

In [66]:
# Metadata columns (excluding the merge key 'frame') that might be duplicated
metadata_cols_to_clean = ['filed', 'company_name', 'end', 'unit', 'form', 'cik']
# Define the possible suffixes added during merges
suffixes = ['_bs', '_is', '_merged', '_cf'] # Ensure all used suffixes are listed

print("\n--- Cleaning up metadata columns ---")

for col in metadata_cols_to_clean:
    # Find all versions of the column (original name or with suffixes)
    col_versions = [c for c in merged_df_final.columns if c == col or any(c == col + s for s in suffixes)]

    if not col_versions:
        # print(f"Metadata column '{col}' not found.")
        continue

    if len(col_versions) == 1:
        # print(f"Metadata column '{col}' has no duplicates.")
        # Ensure the single version is named correctly (without suffix)
        if col_versions[0] != col and col_versions[0] in merged_df_final.columns:
             merged_df_final.rename(columns={col_versions[0]: col}, inplace=True)
        continue

    # print(f"Coalescing column: {col} from versions: {col_versions}")
    # Identify the base column name (could be the first one found, might have a suffix initially)
    base_col_name = col_versions[0]
    # Ensure the target column exists with the correct base name
    if base_col_name != col and base_col_name in merged_df_final.columns:
         merged_df_final.rename(columns={base_col_name: col}, inplace=True)
    elif base_col_name == col and col not in merged_df_final.columns:
         # This case should ideally not happen if col_versions is not empty, but handle defensively
         print(f"Warning: Base column '{col}' expected but not found directly.")
         continue # Skip if the intended target column isn't there after potential rename

    # Use fillna to fill NaNs in the target column 'col' using other versions
    for version in col_versions[1:]:
        if version in merged_df_final.columns: # Check if the source version column exists
            # print(f"  Filling NaNs in '{col}' using '{version}'")
            # Use fillna to populate missing values in 'col' from 'version'
            merged_df_final[col] = merged_df_final[col].fillna(merged_df_final[version])
        # else: # Optional: print if a version column is missing (might happen with outer merges)
            # print(f"  Version column '{version}' not found for filling '{col}'.")


    # Drop the now redundant source columns (all except the target 'col')
    cols_to_drop = [v for v in col_versions if v != col and v in merged_df_final.columns]
    if cols_to_drop:
      # print(f"  Dropping redundant columns: {cols_to_drop}")
      merged_df_final = merged_df_final.drop(columns=cols_to_drop)


--- Cleaning up metadata columns ---


# Ratio Analysis
### Some of the accounts cells are NaN. When computing the ratios that might cause a Divide by Zero error or a NaN error. For those scenarios I will define a function to safely divide by zero 

In [67]:
import pandas as pd
import numpy as np

def calculate_balance_sheet_ratios(df):
    """
    Calculate financial ratios from balance sheet accounts.
    
    Parameters:
    df (DataFrame): DataFrame containing the balance sheet accounts
    
    Returns:
    DataFrame: Original DataFrame with added ratio columns
    """
    # Create a copy of the DataFrame to avoid modifying the original
    result_df = df.copy()
    
    # Define a function to safely calculate ratios, handling division by zero and NaN
    def safe_divide(numerator, denominator):
        # Perform the division
        result = numerator / denominator
        # Replace infinite values (from division by zero) with NaN
        result.replace([np.inf, -np.inf], np.nan, inplace=True)
        return result
    
    # Define a function to check if all required columns exist
    def columns_exist(column_list):
        return all(column in df.columns for column in column_list)
    
    # Define division-based ratios to calculate
    division_ratios = [
        {
            'name': 'Current_Ratio',
            'numerator': 'Assets - Total Current Assets',
            'denominator': 'Liabilities - Total Current Liabilities',
            'description': 'Current Assets / Current Liabilities'
        },
        {
            'name': 'Quick_Ratio',
            'numerator_formula': lambda df: df['Assets - Total Current Assets'] - df['Assets - Inventory'],
            'numerator_columns': ['Assets - Total Current Assets', 'Assets - Inventory'],
            'denominator': 'Liabilities - Total Current Liabilities',
            'description': '(Current Assets - Inventory) / Current Liabilities'
        },
        {
            'name': 'Cash_Ratio',
            'numerator': 'Assets - Cash and Cash Equivalents',
            'denominator': 'Liabilities - Total Current Liabilities',
            'description': 'Cash and Cash Equivalents / Current Liabilities'
        },
        {
            'name': 'Debt_to_Equity_Ratio',
            'numerator': 'Liabilities - Total Liabilities',
            'denominator': 'Equity - Total Stockholders Equity',
            'description': 'Total Liabilities / Total Stockholders Equity'
        },
        {
            'name': 'Debt_to_Assets_Ratio',
            'numerator': 'Liabilities - Total Liabilities',
            'denominator': 'Assets - Total Assets',
            'description': 'Total Liabilities / Total Assets'
        },
        {
            'name': 'Equity_Ratio',
            'numerator': 'Equity - Total Stockholders Equity',
            'denominator': 'Assets - Total Assets',
            'description': 'Total Stockholders Equity / Total Assets'
        }
    ]
    
    # Define subtraction-based metrics
    subtraction_metrics = [
        {
            'name': 'Working_Capital',
            'minuend': 'Assets - Total Current Assets',
            'subtrahend': 'Liabilities - Total Current Liabilities',
            'description': 'Current Assets - Current Liabilities'
        }
    ]
    
    # Track which ratios were calculated and which were skipped
    calculated_ratios = []
    skipped_ratios = []
    
    # Calculate each division-based ratio if the required columns exist
    for ratio in division_ratios:
        if 'numerator_formula' in ratio:
            # Complex numerator requiring multiple columns
            if columns_exist(ratio['numerator_columns'] + [ratio['denominator']]):
                result_df[ratio['name']] = safe_divide(
                    ratio['numerator_formula'](result_df),
                    result_df[ratio['denominator']]
                )
                calculated_ratios.append(ratio['name'])
            else:
                skipped_ratios.append(f"{ratio['name']} ({ratio['description']})")
        else:
            # Simple numerator and denominator
            if columns_exist([ratio['numerator'], ratio['denominator']]):
                result_df[ratio['name']] = safe_divide(
                    result_df[ratio['numerator']],
                    result_df[ratio['denominator']]
                )
                calculated_ratios.append(ratio['name'])
            else:
                skipped_ratios.append(f"{ratio['name']} ({ratio['description']})")
    
    # Calculate each subtraction-based metric if the required columns exist
    for metric in subtraction_metrics:
        if columns_exist([metric['minuend'], metric['subtrahend']]):
            result_df[metric['name']] = result_df[metric['minuend']] - result_df[metric['subtrahend']]
            calculated_ratios.append(metric['name'])
        else:
            skipped_ratios.append(f"{metric['name']} ({metric['description']})")
    
    # Print summary
    if calculated_ratios:
        print(f"Successfully calculated: {', '.join(calculated_ratios)}")
    if skipped_ratios:
        print(f"Skipped due to missing columns: {', '.join(skipped_ratios)}")
    
    return result_df

# Example usage:
# Assuming 'balance_sheet' is a DataFrame containing the balance sheet accounts
financial_data_with_ratios = calculate_balance_sheet_ratios(balance_sheet)
financial_data_with_ratios.head()

Successfully calculated: Current_Ratio, Quick_Ratio, Cash_Ratio, Debt_to_Equity_Ratio, Debt_to_Assets_Ratio, Equity_Ratio, Working_Capital


,filed,company_name,end,unit,form,frame,cik,Assets - Cash and Cash Equivalents,Assets - Short-term Investments,"Assets - Accounts Receivable, Net (Current)",...,Validation - Current Liabilities Difference,Validation - A = L+E Difference,Validation - Components = Total L+E Difference,Current_Ratio,Quick_Ratio,Cash_Ratio,Debt_to_Equity_Ratio,Debt_to_Assets_Ratio,Equity_Ratio,Working_Capital
48,2019-04-26,"AMAZON.COM, INC.",2018-03-31,USD,10-Q,CY2017,1018724,1.667600e+10,8.287000e+09,1.202600e+10,...,4.799200e+10,0.0,0.000000e+00,1.057946,0.769882,0.347091,3.016210,0.751009,0.248991,2.784000e+09
52,2019-02-01,"AMAZON.COM, INC.",2018-12-31,USD,10-K,CY2018,1018724,3.175000e+10,9.500000e+09,1.667700e+10,...,6.702000e+10,0.0,-1.183500e+11,1.098112,0.846997,0.464242,0.017199,0.004605,0.267750,6.710000e+09
53,2019-04-26,"AMAZON.COM, INC.",2019-03-31,USD,10-Q,CY2019Q1,1018724,2.311500e+10,1.390500e+10,1.597900e+10,...,6.208700e+10,0.0,0.000000e+00,1.090054,0.832075,0.362901,2.679033,0.728189,0.271811,5.736000e+09
55,2019-07-26,"AMAZON.COM, INC.",2019-06-30,USD,10-Q,CY2019Q2,1018724,2.261600e+10,1.884700e+10,1.674700e+10,...,6.804200e+10,0.0,0.000000e+00,1.102070,0.835414,0.324579,2.606246,0.722703,0.277297,7.112000e+09
56,2019-10-25,"AMAZON.COM, INC.",2019-09-30,USD,10-Q,CY2019Q3,1018724,2.325500e+10,2.014600e+10,1.688700e+10,...,6.929500e+10,0.0,0.000000e+00,1.095902,0.835755,0.322377,2.523377,0.716181,0.283819,6.918000e+09


In [68]:
financial_data_with_ratios.head(30)

,filed,company_name,end,unit,form,frame,cik,Assets - Cash and Cash Equivalents,Assets - Short-term Investments,"Assets - Accounts Receivable, Net (Current)",...,Validation - Current Liabilities Difference,Validation - A = L+E Difference,Validation - Components = Total L+E Difference,Current_Ratio,Quick_Ratio,Cash_Ratio,Debt_to_Equity_Ratio,Debt_to_Assets_Ratio,Equity_Ratio,Working_Capital
48,2019-04-26,"AMAZON.COM, INC.",2018-03-31,USD,10-Q,CY2017,1018724,1.667600e+10,8.287000e+09,1.202600e+10,...,4.799200e+10,0.0,0.000000e+00,1.057946,0.769882,0.347091,3.016210,0.751009,0.248991,2.784000e+09
52,2019-02-01,"AMAZON.COM, INC.",2018-12-31,USD,10-K,CY2018,1018724,3.175000e+10,9.500000e+09,1.667700e+10,...,6.702000e+10,0.0,-1.183500e+11,1.098112,0.846997,0.464242,0.017199,0.004605,0.267750,6.710000e+09
53,2019-04-26,"AMAZON.COM, INC.",2019-03-31,USD,10-Q,CY2019Q1,1018724,2.311500e+10,1.390500e+10,1.597900e+10,...,6.208700e+10,0.0,0.000000e+00,1.090054,0.832075,0.362901,2.679033,0.728189,0.271811,5.736000e+09
55,2019-07-26,"AMAZON.COM, INC.",2019-06-30,USD,10-Q,CY2019Q2,1018724,2.261600e+10,1.884700e+10,1.674700e+10,...,6.804200e+10,0.0,0.000000e+00,1.102070,0.835414,0.324579,2.606246,0.722703,0.277297,7.112000e+09
56,2019-10-25,"AMAZON.COM, INC.",2019-09-30,USD,10-Q,CY2019Q3,1018724,2.325500e+10,2.014600e+10,1.688700e+10,...,6.929500e+10,0.0,0.000000e+00,1.095902,0.835755,0.322377,2.523377,0.716181,0.283819,6.918000e+09
57,2020-01-31,"AMAZON.COM, INC.",2019-12-31,USD,10-K,CY2019,1018724,3.609200e+10,1.892900e+10,2.081600e+10,...,8.650700e+10,0.0,-1.625480e+11,1.097048,0.863629,0.411014,0.010313,0.002841,0.275519,8.522000e+09
58,2021-04-30,"AMAZON.COM, INC.",2020-03-31,USD,10-Q,CY2019,1018724,2.720100e+10,2.209100e+10,1.783600e+10,...,7.840000e+10,0.0,0.000000e+00,1.078709,0.842142,0.341245,2.389478,0.704969,0.295031,6.274000e+09
60,2021-07-30,"AMAZON.COM, INC.",2020-06-30,USD,10-Q,CY2020Q2,1018724,3.746600e+10,3.392500e+10,1.991800e+10,...,9.274200e+10,0.0,0.000000e+00,1.181179,0.972448,0.399016,2.503608,0.714580,0.285420,1.701200e+10
61,2020-10-30,"AMAZON.COM, INC.",2020-09-30,USD,10-Q,CY2020Q3,1018724,2.993000e+10,3.847200e+10,2.083200e+10,...,1.017570e+11,0.0,0.000000e+00,1.108496,0.875599,0.293685,2.408988,0.706658,0.293342,1.105700e+10
62,2021-02-03,"AMAZON.COM, INC.",2020-12-31,USD,10-K,CY2020,1018724,4.212200e+10,4.227400e+10,2.454200e+10,...,1.252300e+11,0.0,-2.268980e+11,1.050227,0.861954,0.333283,0.009561,0.002780,0.290802,6.348000e+09


In [69]:
income_statement.columns

Index(['filed', 'company_name', 'end', 'unit', 'form', 'frame', 'cik',
       'Revenue - Total Revenue', 'Cost - Total Cost of Revenue',
       'Operating Expenses - Sales and Marketing',
       'Operating Expenses - General and Administrative',
       'Operating Expenses - Selling, General and Administrative',
       'Operating Expenses - Amortization of Intangible Assets',
       'Operating - Operating Income', 'Other - Interest Income',
       'Other - Interest Expense',
       'Other - Foreign Currency Transaction Gain/Loss',
       'Other - Other Income/Expense, Net', 'Tax - Income Tax Expense/Benefit',
       'Net Income - Net Income', 'EPS - Basic EPS', 'EPS - Diluted EPS',
       'Gross - Gross Profit (calculated)',
       'Income Before Tax - Income Before Income Taxes (calculated)'],
      dtype='object')

In [70]:
import pandas as pd
import numpy as np

def calculate_income_statement_ratios(df):
    """
    Calculate financial ratios from income statement accounts.
    
    Parameters:
    df (DataFrame): DataFrame containing the income statement accounts
    
    Returns:
    DataFrame: Original DataFrame with added ratio columns
    """
    # Create a copy of the DataFrame to avoid modifying the original
    result_df = df.copy()
    
    # Define a function to safely calculate ratios, handling division by zero and NaN
    def safe_divide(numerator, denominator):
        # Perform the division
        result = numerator / denominator
        # Replace infinite values (from division by zero) with NaN
        result.replace([np.inf, -np.inf], np.nan, inplace=True)
        return result
    
    # Define a function to check if all required columns exist
    def columns_exist(column_list):
        return all(column in df.columns for column in column_list)
    
    # Define ratios to calculate
    ratios = [
        {
            'name': 'Gross_Profit_Margin',
            'numerator': 'Gross - Gross Profit',
            'denominator': 'Revenue - Total Revenue',
            'description': 'Gross Profit / Revenue'
        },
        {
            'name': 'Operating_Profit_Margin',
            'numerator': 'Operating - Operating Income',
            'denominator': 'Revenue - Total Revenue',
            'description': 'Operating Income / Revenue'
        },
        {
            'name': 'Net_Profit_Margin',
            'numerator': 'Net Income - Net Income',
            'denominator': 'Revenue - Total Revenue',
            'description': 'Net Income / Revenue'
        },
        {
            'name': 'Interest_Coverage_Ratio',
            'numerator': 'Operating - Operating Income',
            'denominator': 'Other - Interest Expense',
            'description': 'Operating Income / Interest Expense'
        }
    ]
    
    # Track which ratios were calculated and which were skipped
    calculated_ratios = []
    skipped_ratios = []
    
    # Calculate each ratio if the required columns exist
    for ratio in ratios:
        if columns_exist([ratio['numerator'], ratio['denominator']]):
            result_df[ratio['name']] = safe_divide(
                result_df[ratio['numerator']],
                result_df[ratio['denominator']]
            )
            calculated_ratios.append(ratio['name'])
        else:
            skipped_ratios.append(f"{ratio['name']} ({ratio['description']})")
    
    # Print summary
    if calculated_ratios:
        print(f"Successfully calculated: {', '.join(calculated_ratios)}")
    if skipped_ratios:
        print(f"Skipped due to missing columns: {', '.join(skipped_ratios)}")
    
    return result_df

# Example usage:
# Assuming 'income_data' is a DataFrame containing the income statement accounts
income_data_with_ratios = calculate_income_statement_ratios(income_statement)
income_data_with_ratios.head(30)

Successfully calculated: Operating_Profit_Margin, Net_Profit_Margin, Interest_Coverage_Ratio
Skipped due to missing columns: Gross_Profit_Margin (Gross Profit / Revenue)


,filed,company_name,end,unit,form,frame,cik,Revenue - Total Revenue,Cost - Total Cost of Revenue,Operating Expenses - Sales and Marketing,...,"Other - Other Income/Expense, Net",Tax - Income Tax Expense/Benefit,Net Income - Net Income,EPS - Basic EPS,EPS - Diluted EPS,Gross - Gross Profit (calculated),Income Before Tax - Income Before Income Taxes (calculated),Operating_Profit_Margin,Net_Profit_Margin,Interest_Coverage_Ratio
48,2019-04-26,"AMAZON.COM, INC.",2018-03-31,USD,10-Q,CY2017,1018724,5.104200e+10,3.073500e+10,2.699000e+09,...,-1.100000e+07,2.870000e+08,1.629000e+09,3.36,3.27,2.030700e+10,1.916000e+09,0.037753,0.031915,5.839394
52,2019-02-01,"AMAZON.COM, INC.",2018-12-31,USD,10-K,CY2018,1018724,2.328870e+11,1.391560e+11,1.381400e+10,...,-1.160000e+09,1.197000e+09,1.007300e+10,20.68,20.14,9.373100e+10,1.126100e+10,0.053335,0.043253,8.765702
53,2019-04-26,"AMAZON.COM, INC.",2019-03-31,USD,10-Q,CY2019Q1,1018724,5.970000e+10,3.392000e+10,3.664000e+09,...,-1.900000e+07,8.360000e+08,3.561000e+09,7.24,7.09,2.578000e+10,4.401000e+09,0.074037,0.059648,12.076503
55,2019-07-26,"AMAZON.COM, INC.",2019-06-30,USD,10-Q,CY2019Q2,1018724,6.340400e+10,3.633700e+10,4.291000e+09,...,-1.950000e+08,2.570000e+08,2.625000e+09,5.32,5.22,2.706700e+10,2.889000e+09,0.048640,0.041401,8.052219
56,2019-10-25,"AMAZON.COM, INC.",2019-09-30,USD,10-Q,CY2019Q3,1018724,6.998100e+10,4.130200e+10,4.752000e+09,...,-5.250000e+08,4.940000e+08,2.134000e+09,4.31,4.23,2.867900e+10,2.632000e+09,0.045112,0.030494,7.972222
57,2020-01-31,"AMAZON.COM, INC.",2019-12-31,USD,10-K,CY2019,1018724,2.805220e+11,1.655360e+11,1.887800e+10,...,-5.650000e+08,2.374000e+09,1.158800e+10,23.46,23.01,1.149860e+11,1.397600e+10,0.051836,0.041309,9.088125
58,2021-04-30,"AMAZON.COM, INC.",2020-03-31,USD,10-Q,CY2019,1018724,7.545200e+10,4.425700e+10,4.828000e+09,...,-6.060000e+08,7.440000e+08,2.535000e+09,5.09,5.01,3.119500e+10,3.383000e+09,0.052868,0.033598,9.922886
60,2021-07-30,"AMAZON.COM, INC.",2020-06-30,USD,10-Q,CY2020Q2,1018724,8.891200e+10,5.266000e+10,4.345000e+09,...,3.780000e+08,9.840000e+08,5.243000e+09,10.50,10.30,3.625200e+10,6.221000e+09,0.065717,0.058968,14.498759
61,2020-10-30,"AMAZON.COM, INC.",2020-09-30,USD,10-Q,CY2020Q3,1018724,9.614500e+10,5.710600e+10,5.434000e+09,...,6.150000e+08,5.690000e+08,6.331000e+09,12.63,12.37,3.903900e+10,6.809000e+09,0.064424,0.065848,14.471963
62,2021-02-03,"AMAZON.COM, INC.",2020-12-31,USD,10-K,CY2020,1018724,3.860640e+11,2.333070e+11,2.200800e+10,...,1.279000e+09,2.863000e+09,2.133100e+10,2.13,2.09,1.527570e+11,2.417800e+10,0.059314,0.055252,13.903461


#### There are some ratios that are not well computed because they need computation accross different time period but it will not work for the first period so you need a way out of it. You could shift the whole dataframe or you could start the computation on the second one. 

- Note: Ratios requiring averages (like ROA, ROE, Turnovers, DSO, DIO, DPO) are calculated
- using end-of-period values here for simplicity. For more accurate analysis,
- calculate average balances: (balance_start_period + balance_end_period) / 2.
- This requires having data from the previous period (e.g., using df.shift()).

### In what follows I have defined different functions that compute Liquidity Ratios